In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, precision_score, recall_score

2024-09-25 13:13:11.264822: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-25 13:13:11.277859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 13:13:11.296654: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 13:13:11.296678: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 13:13:11.308952: I tensorflow/core/platform/cpu_feature_gua

In [34]:
pd.set_option('display.max_columns', None)
train=pd.read_csv('../juhye/dd4_train.csv')
test=pd.read_csv('../juhye/dd4_test.csv')

train

,subject_id,hadm_id,chartdate,SBP_min,SBP_max,SBP_mean,DBP_min,DBP_max,DBP_mean,MAP_min,MAP_max,MAP_mean,HR_min,HR_max,HR_mean,SpO2_min,SpO2_max,SpO2_mean,Creatinine_mean,WBC_mean,Hemoglobin_mean,Platelet_mean,Potassium_mean,Chloride_mean,AST_mean,ALT_mean,Bilirubin_mean,Glucose_mean,Sodium_mean,gender,anchor_age,race,medicine,weight,height,diagnose
0,16230775,20002506,2132-03-19,98.0,114.0,107.055556,53.0,92.0,64.500000,72.0,99.0,77.055556,62.0,110.0,71.555556,95.0,100.0,97.944444,0.70,10.70,13.80,223.0,4.100000,101.000000,28.0,17.0,1.1,100.0,141.500000,0,24,UNKNOWN,1,60.0,NaN,0
1,16230775,20002506,2132-03-20,105.0,136.0,118.400000,61.0,80.0,70.200000,74.0,92.0,83.333333,56.0,98.0,68.600000,96.0,100.0,98.200000,0.70,8.00,13.00,174.5,3.950000,105.000000,NaN,NaN,NaN,96.5,143.333333,0,24,UNKNOWN,1,60.0,NaN,0
2,16230775,20002506,2132-03-21,110.0,133.0,123.562500,65.0,86.0,73.312500,80.0,98.0,87.250000,56.0,81.0,64.625000,96.0,100.0,98.750000,0.70,9.20,12.75,179.5,3.900000,101.000000,NaN,NaN,NaN,114.0,140.000000,0,24,UNKNOWN,1,60.0,NaN,0
3,16230775,20002506,2132-03-22,103.0,129.0,118.529412,54.0,85.0,74.941176,70.0,97.0,87.187500,54.0,69.0,60.500000,97.0,100.0,98.684211,0.60,8.90,12.10,177.0,3.700000,95.000000,NaN,NaN,NaN,131.0,136.000000,0,24,UNKNOWN,1,60.0,NaN,0
4,16230775,20002506,2132-03-23,96.0,131.0,117.250000,68.0,84.0,77.000000,79.0,94.0,88.750000,52.0,81.0,60.500000,95.0,100.0,98.928571,0.70,7.50,12.80,205.0,3.400000,97.000000,NaN,NaN,NaN,106.0,137.000000,0,24,UNKNOWN,1,60.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106980,15129243,29999098,2128-06-16,99.0,218.0,158.551724,47.0,92.0,64.793103,66.0,139.0,94.206897,65.0,96.0,75.965517,95.0,98.0,96.448276,0.80,10.80,9.50,23.0,3.866667,107.666667,NaN,NaN,NaN,232.0,141.000000,0,76,WHITE,1,91.6,167.64,0
106981,15129243,29999098,2128-06-17,139.0,189.0,164.333333,53.0,81.0,68.666667,71.0,114.0,94.208333,57.0,88.0,72.500000,95.0,100.0,96.875000,0.80,13.30,8.10,23.0,3.933333,105.333333,NaN,NaN,NaN,273.5,139.000000,0,76,WHITE,1,91.6,167.64,0
106982,15129243,29999098,2128-06-18,115.0,200.0,150.166667,43.0,98.0,62.583333,60.0,122.0,83.333333,60.0,94.0,74.833333,93.0,100.0,96.666667,0.90,15.60,7.10,37.0,3.900000,106.500000,NaN,NaN,NaN,124.5,139.500000,0,76,WHITE,1,91.6,167.64,0
106983,15129243,29999098,2128-06-19,86.0,127.0,107.500000,42.0,69.0,53.208333,52.0,81.0,64.833333,46.0,124.0,84.320000,93.0,100.0,96.760000,1.15,23.95,6.55,80.0,4.100000,123.500000,NaN,NaN,NaN,346.0,164.250000,0,76,WHITE,1,91.6,167.64,0


In [3]:
train.isna().sum()

subject_id             0
hadm_id                0
chartdate              0
SBP_min                1
SBP_max                1
SBP_mean               1
DBP_min                6
DBP_max                6
DBP_mean               6
MAP_min               37
MAP_max               37
MAP_mean              37
HR_min                 8
HR_max                 8
HR_mean                8
SpO2_min              87
SpO2_max              87
SpO2_mean             87
Creatinine_mean        0
WBC_mean            1830
Hemoglobin_mean     1778
Platelet_mean       1854
Potassium_mean       433
Chloride_mean        214
AST_mean           68227
ALT_mean           68120
Bilirubin_mean     68346
Glucose_mean         177
Sodium_mean           57
gender                 0
anchor_age             0
race                   0
medicine               0
weight                 0
height             16350
diagnose               0
dtype: int64

In [4]:
def detect_aki(df):
    # 'AKI' 컬럼 초기화
    df['AKI'] = 0
    
    # hadm_id별로 그룹화
    for hadm_id, group in df.groupby('hadm_id'):
        group = group.sort_values(by='chartdate')  # 날짜 기준 정렬
        for i in range(len(group)):
            # 현재 Creatinine_mean
            current_creatinine = group.iloc[i]['Creatinine_mean']
            
            # 2일 전 Creatinine_mean
            if i >= 2:
                two_days_ago_creatinine = group.iloc[i - 2]['Creatinine_mean']
                if current_creatinine - two_days_ago_creatinine >= 0.3:
                    df.loc[group.index[i], 'AKI'] = 1
            
            # 7일 전 Creatinine_mean
            if i >= 7:
                seven_days_ago_creatinine = group.iloc[i - 7]['Creatinine_mean']
                if current_creatinine >= seven_days_ago_creatinine * 1.5:
                    df.loc[group.index[i], 'AKI'] = 1

    return df

# # 예시 데이터프레임 생성
# data = {
#     'hadm_id': [1, 1, 1, 1, 1, 1, 1, 1],
#     'chartdate': pd.date_range(start='2024-01-01', periods=8),
#     'Creatinine_mean': [1.0, 1.1, 1.2, 1.5, 1.6, 1.7, 1.8, 2.0]
# }

train=detect_aki(train)
test=detect_aki(test)
test.head()
# train, test 에 AKI 발병 여부 칼럼 추가함

In [5]:
# train, test에서 nan값 이 있는 칼럼들에 대해서 nan 비율을 보도록 해보자!
from itertools import zip_longest
res_train = train.isna().mean()
res_train_non_zero = res_train[res_train>0]
res_test = test.isna().mean()
res_test_non_zero = res_test[res_test>0]
# 옆에 나란히 출력하기
for (train_index, train_value), (test_index, test_value) in zip_longest(res_train_non_zero.items(), res_test_non_zero.items(), fillvalue=(None, None)):
    print(f"{train_index}: {train_value:.6f} \t\t {test_index}: {test_value:.6f}")
    
# train에서 nan 비율이 60% 가 넘는 것들이 있음 : AST_mean,ALT_mean,Bilirubin_mean
# -> 위에 세 녀석은 학습에 사용하지 않도록 한다.(test에서 nan값이 60%가 넘는건 실제 상황에서는 관찰 할 수 없는 경우라서 test 의 정보들은 데이터 전처리에 사용이 되면 안됨)
drop_col = ['AST_mean','ALT_mean','Bilirubin_mean']
train.drop(columns=drop_col,axis = 1, inplace=True)
test.drop(columns=drop_col,axis = 1, inplace=True)

res_train = train.isna().mean()
res_train_non_zero = res_train[res_train>0]
res_test = test.isna().mean()
res_test_non_zero = res_test[res_test>0]

SBP_min: 0.000009 		 SBP_min: 0.000022
SBP_max: 0.000009 		 SBP_max: 0.000022
SBP_mean: 0.000009 		 SBP_mean: 0.000022
DBP_min: 0.000056 		 DBP_min: 0.000109
DBP_max: 0.000056 		 DBP_max: 0.000109
DBP_mean: 0.000056 		 DBP_mean: 0.000109
MAP_min: 0.000346 		 MAP_min: 0.000391
MAP_max: 0.000346 		 MAP_max: 0.000391
MAP_mean: 0.000346 		 MAP_mean: 0.000391
HR_min: 0.000075 		 HR_min: 0.000065
HR_max: 0.000075 		 HR_max: 0.000065
HR_mean: 0.000075 		 HR_mean: 0.000065
SpO2_min: 0.000813 		 SpO2_min: 0.000651
SpO2_max: 0.000813 		 SpO2_max: 0.000651
SpO2_mean: 0.000813 		 SpO2_mean: 0.000651
WBC_mean: 0.017105 		 WBC_mean: 0.018307
Hemoglobin_mean: 0.016619 		 Hemoglobin_mean: 0.018025
Platelet_mean: 0.017330 		 Platelet_mean: 0.019306
Potassium_mean: 0.004047 		 Potassium_mean: 0.003648
Chloride_mean: 0.002000 		 Chloride_mean: 0.001998
AST_mean: 0.637725 		 AST_mean: 0.636445
ALT_mean: 0.636725 		 ALT_mean: 0.635598
Bilirubin_mean: 0.638837 		 Bilirubin_mean: 0.637313
Glucose_mean: 0.001

In [6]:
res_train = train.isna().mean()
res_train_non_zero = res_train[res_train>0]
res_test = test.isna().mean()
res_test_non_zero = res_test[res_test>0]
for (train_index, train_value), (test_index, test_value) in zip_longest(res_train_non_zero.items(), res_test_non_zero.items(), fillvalue=(None, None)):
    print(f"{train_index}: {train_value:.6f} \t\t {test_index}: {test_value:.6f}")

SBP_min: 0.000009 		 SBP_min: 0.000022
SBP_max: 0.000009 		 SBP_max: 0.000022
SBP_mean: 0.000009 		 SBP_mean: 0.000022
DBP_min: 0.000056 		 DBP_min: 0.000109
DBP_max: 0.000056 		 DBP_max: 0.000109
DBP_mean: 0.000056 		 DBP_mean: 0.000109
MAP_min: 0.000346 		 MAP_min: 0.000391
MAP_max: 0.000346 		 MAP_max: 0.000391
MAP_mean: 0.000346 		 MAP_mean: 0.000391
HR_min: 0.000075 		 HR_min: 0.000065
HR_max: 0.000075 		 HR_max: 0.000065
HR_mean: 0.000075 		 HR_mean: 0.000065
SpO2_min: 0.000813 		 SpO2_min: 0.000651
SpO2_max: 0.000813 		 SpO2_max: 0.000651
SpO2_mean: 0.000813 		 SpO2_mean: 0.000651
WBC_mean: 0.017105 		 WBC_mean: 0.018307
Hemoglobin_mean: 0.016619 		 Hemoglobin_mean: 0.018025
Platelet_mean: 0.017330 		 Platelet_mean: 0.019306
Potassium_mean: 0.004047 		 Potassium_mean: 0.003648
Chloride_mean: 0.002000 		 Chloride_mean: 0.001998
Glucose_mean: 0.001654 		 Glucose_mean: 0.001498
Sodium_mean: 0.000533 		 Sodium_mean: 0.000304
height: 0.152825 		 height: 0.159051


In [7]:
train['AKI'].value_counts()/train.shape[0]

AKI
0    0.878179
1    0.121821
Name: count, dtype: float64

In [8]:
test['AKI'].value_counts()/test.shape[0]
# trian과 test둘다 타겟 칼럼인 AKI의 0과 1의 비율이 비슷하다 
# 응 근데 unbalance해서 업샘플링을 하거나 모델에서 파라미터 조절을 해야할듯(scale_pos_weight 같은 파라미터들)

AKI
0    0.877172
1    0.122828
Name: count, dtype: float64

In [9]:
import pandas as pd

def fill_missing_values_with_interpolation(df, value_column, group_column='hadm_id', method='linear'):
    """
    각 입원 아이디(hadm_id)별로 그룹화하여, 해당 검사 수치의 NaN 값을 보간법으로 채움.
    
    Args:
    df (pd.DataFrame): 데이터프레임.
    value_column (str): 보간법을 적용할 검사 수치 칼럼 이름.
    group_column (str): 그룹화할 기준 칼럼 이름 (기본값은 'hadm_id').
    method (str): 보간법 종류 (기본값은 'linear').
    
    Returns:
    pd.DataFrame: NaN이 보간법으로 채워진 데이터프레임.
    """
    
    def interpolate_group(group):
        # 그룹을 chartdate 기준으로 정렬
        group = group.sort_values(by='chartdate')
        
        # 해당 그룹의 값이 NaN이 아닌 경우에만 보간법 적용
        if group[value_column].isnull().all():
            # 모든 값이 NaN인 경우는 보간법을 적용하지 않고 그대로 NaN 유지
            return group
        else:
            # 선형 보간법으로 NaN 값 채우기
            group[value_column] = group[value_column].interpolate(method=method, limit_direction='both')
            return group
    
    # 각 그룹(hadm_id)별로 보간법 적용
    df_filled = df.groupby(group_column).apply(interpolate_group)
    
    # 인덱스 재정렬
    df_filled = df_filled.reset_index(drop=True)
    
    return df_filled

In [10]:
# print(train.columns)
# all_columns=train.columns
# not_num=['hadm_id','medicine', 'diagnose','gender','AKI']  # 선형 보간 처리안할 칼럼
# num_col = list(set(all_columns) - set(category))
# ---------------------------------------------------------
# 선형 보간 처리할 칼럼 뽑아주기
col_nan = res_train_non_zero.index.tolist() # nan 값이 있는 칼럼
print(col_nan)
col_nan.remove('height') # 키는 선형 회귀로 예측해서 nan값 채워줄거임
print(col_nan)

['SBP_min', 'SBP_max', 'SBP_mean', 'DBP_min', 'DBP_max', 'DBP_mean', 'MAP_min', 'MAP_max', 'MAP_mean', 'HR_min', 'HR_max', 'HR_mean', 'SpO2_min', 'SpO2_max', 'SpO2_mean', 'WBC_mean', 'Hemoglobin_mean', 'Platelet_mean', 'Potassium_mean', 'Chloride_mean', 'Glucose_mean', 'Sodium_mean', 'height']
['SBP_min', 'SBP_max', 'SBP_mean', 'DBP_min', 'DBP_max', 'DBP_mean', 'MAP_min', 'MAP_max', 'MAP_mean', 'HR_min', 'HR_max', 'HR_mean', 'SpO2_min', 'SpO2_max', 'SpO2_mean', 'WBC_mean', 'Hemoglobin_mean', 'Platelet_mean', 'Potassium_mean', 'Chloride_mean', 'Glucose_mean', 'Sodium_mean']


In [11]:
# 입원 아이디 기준으로 보간법 써서 결측치 채우기
for c in col_nan:
    train = fill_missing_values_with_interpolation(train, c)
    
for c in col_nan:
    test = fill_missing_values_with_interpolation(test,c)

/tmp/ipykernel_40887/2631295187.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filled = df.groupby(group_column).apply(interpolate_group)
/tmp/ipykernel_40887/2631295187.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filled = df.groupby(group_column).apply(interpolate_group)
/tmp/ipykernel_40887/2631295187.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. 

In [12]:
res_train = train.isna().mean()
res_train_non_zero = res_train[res_train>0]
res_test = test.isna().mean()
res_test_non_zero = res_test[res_test>0]
for (train_index, train_value), (test_index, test_value) in zip_longest(res_train_non_zero.items(), res_test_non_zero.items(), fillvalue=(None, None)):
    print(f"{train_index}: {train_value:.6f} \t\t {test_index}: {test_value:.6f}")
# 입원 id 기준으로 보간법 써서 nan 값 채워도 여전히 nan값이 남아있는 녀석들이 있다.

WBC_mean: 0.000103 		 WBC_mean: 0.000130
Platelet_mean: 0.000103 		 Hemoglobin_mean: 0.000130
Chloride_mean: 0.000093 		 Platelet_mean: 0.000130
height: 0.152825 		 height: 0.159051


In [13]:
interpol_train_col = res_train_non_zero.index.tolist()
print(interpol_train_col)
interpol_train_col.remove('height')
print(interpol_train_col)
# train 기준으로 여전히 nan이 남아있는 칼럼들은 subject_id기준으로 보간법 써서 nan값 채워주겠음
print('\n')
# 보간법은 입원 및 환자 id 기준으로 보간법 쓰는거니까 train 기준으로 nan값을 채울 필요가 없다고 판단하고
# test도 test 기준으로 보간법 쓰도록 하겠음
# height 및 weight 예측할 때는 test의 height 와 weight의 nan 값은 train 기준으로 학습한 모델을 사용해서 예측해야 오류가 없다.
interpol_test_col = res_test_non_zero.index.tolist()
print(interpol_test_col)
interpol_test_col.remove('height')
print(interpol_test_col)

['WBC_mean', 'Platelet_mean', 'Chloride_mean', 'height']
['WBC_mean', 'Platelet_mean', 'Chloride_mean']


['WBC_mean', 'Hemoglobin_mean', 'Platelet_mean', 'height']
['WBC_mean', 'Hemoglobin_mean', 'Platelet_mean']


In [14]:
# 입원 아이디 기준으로 보간법 써서 결측치 채우기
for c in interpol_train_col:
    train = fill_missing_values_with_interpolation(train, c,group_column = 'subject_id')
    
for c in interpol_test_col:
    test = fill_missing_values_with_interpolation(test,c,group_column = 'subject_id')

/tmp/ipykernel_40887/2631295187.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filled = df.groupby(group_column).apply(interpolate_group)
/tmp/ipykernel_40887/2631295187.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filled = df.groupby(group_column).apply(interpolate_group)
/tmp/ipykernel_40887/2631295187.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. 

In [15]:
res_train = train.isna().mean()
res_train_non_zero = res_train[res_train>0]
res_test = test.isna().mean()
res_test_non_zero = res_test[res_test>0]
for (train_index, train_value), (test_index, test_value) in zip_longest(res_train_non_zero.items(), res_test_non_zero.items(), fillvalue=(None, None)):
    print(f"{train_index}: {train_value:.6f} \t\t {test_index}: {test_value:.6f}")

WBC_mean: 0.000047 		 WBC_mean: 0.000130
Platelet_mean: 0.000047 		 Hemoglobin_mean: 0.000130
Chloride_mean: 0.000093 		 Platelet_mean: 0.000130
height: 0.152825 		 height: 0.159051


In [16]:
# 입원 id 기준으로 보간법 써서 채우고, subject_id 기준으로 보간법 써서 채워도 채워지지 않은 nan이 있는 칼럼은
# 그 환자 자체가 그 칼럼에 해당하는 수치를 측정한적이 없다는 것을 의미 -> 그 환자는 그냥 row 자체를 제거하기로 한다
del_train_subject = []
del_train_subject.extend(list(set(train[train['WBC_mean'].isna()]['subject_id'])))
del_train_subject.extend(list(set(train[train['Platelet_mean'].isna()]['subject_id'])))
del_train_subject.extend(list(set(train[train['Chloride_mean'].isna()]['subject_id'])))
del_train_subject = list(set(del_train_subject))
print(del_train_subject)
train = train[~train['subject_id'].isin(del_train_subject)]

del_test_subject = []
del_test_subject.extend(list(set(test[test['WBC_mean'].isna()]['subject_id'])))
del_test_subject.extend(list(set(test[test['Platelet_mean'].isna()]['subject_id'])))
del_test_subject.extend(list(set(test[test['Hemoglobin_mean'].isna()]['subject_id'])))
del_test_subject = list(set(del_test_subject))
print(del_test_subject)
test = test[~test['subject_id'].isin(del_test_subject)]

# 이제 nan 값은 height에만 있음
res_train = train.isna().mean()
res_train_non_zero = res_train[res_train>0]
res_test = test.isna().mean()
res_test_non_zero = res_test[res_test>0]
for (train_index, train_value), (test_index, test_value) in zip_longest(res_train_non_zero.items(), res_test_non_zero.items(), fillvalue=(None, None)):
    print(f"{train_index}: {train_value:.6f} \t\t {test_index}: {test_value:.6f}")

[17594325, 13913422, 15635581]
[17926759]
height: 0.152760 		 height: 0.159072


## 키 채우기
- 키는 성별과 나이(보호변수) 와 몸무게를 이용해서 예측할건데, aki 여부 예측 모델에서는 보호변수인 나이와 성별은 학습에 넣지 않을거임
- 키는 성별과 나이와 상관관계가 클것으로 예상되어서 예측에 보호변수를 사용하긴 할거임

In [17]:
height_predict_train = train[['subject_id','anchor_age','gender','weight','height']]
print(len(set(height_predict_train['subject_id'])))
height_predict_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
print(len(set(height_predict_train['subject_id'])))
height_predict_train # 키 예측 모델 학습 및 예측에 사용할 칼럼

10460
10460


/tmp/ipykernel_40887/2141692712.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  height_predict_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)


,subject_id,anchor_age,gender,weight,height
0,10001884,68,1,65.0,157.48
9,10002348,77,1,46.5,NaN
17,10002428,80,1,55.0,149.86
42,10003400,72,1,99.5,167.64
74,10003637,57,0,81.0,175.26
...,...,...,...,...,...
106940,19997752,66,1,86.0,170.18
106946,19998591,52,1,65.8,157.48
106966,19999068,63,0,49.5,170.18
106972,19999287,71,1,87.6,165.10


In [18]:
height_predict_test = test[['subject_id','anchor_age','gender','weight','height']]
print(len(set(height_predict_test['subject_id'])))
height_predict_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
print(len(set(height_predict_test['subject_id'])))
height_predict_test # test데이터셋은 train을 가지고 만든 예측 모델로 nan값 채울거임

4484
4484


/tmp/ipykernel_40887/1564059634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  height_predict_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)


,subject_id,anchor_age,gender,weight,height
0,10002155,80,1,21.1,NaN
7,10004235,47,0,130.0,182.88
12,10004606,64,1,57.1,162.56
18,10004733,51,0,109.0,180.34
27,10005606,38,0,95.0,177.80
...,...,...,...,...,...
45994,19996673,72,1,85.5,165.10
45999,19997448,52,1,85.0,160.02
46004,19997561,62,1,110.0,152.40
46011,19997760,50,0,113.4,182.88


In [19]:
height_predict_train.loc[(height_predict_train['weight'] <= 12) | (height_predict_train['weight'] > 508), 'weight'] = np.nan
# 12키로 이하, 508키로(2006년 기네스북 기준 가장 뚱뚱한 사람의 몸무게) 초과인 사람들은 잘못 기록된 이상치로 보고 nan값으로바꿔주겠음
height_predict_test.loc[(height_predict_test['weight'] <= 12) | (height_predict_test['weight'] > 508), 'weight'] = np.nan
print(height_predict_train.isna().sum())
# 몸무게는 이상치같은 데이터 nan 값 처리해줬더니 nan 이 12개 밖에 없어서 그냥 train과 test 원래 데이터에서 weight가 이상치 범위에 있는 환자들은 그냥 row 자체를 제거하겠다
train = train[(train['weight']>12)&(train['weight']<=508)]
test = test[(test['weight']>12)&(test['weight']<=508)]

height_predict_train = train[['subject_id','anchor_age','gender','weight','height']]
print(len(set(height_predict_train['subject_id'])))
height_predict_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
print(len(set(height_predict_train['subject_id'])))
# height_predict_train # 키 예측 모델 학습 및 예측에 사용할 칼럼
height_predict_test = test[['subject_id','anchor_age','gender','weight','height']]
print(len(set(height_predict_test['subject_id'])))
height_predict_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
print(len(set(height_predict_test['subject_id'])))
# height_predict_test # test데이터셋은 train을 가지고 만든 예측 모델로 nan값 채울거임

subject_id       0
anchor_age       0
gender           0
weight          12
height        2076
dtype: int64
10448
10448
4481
4481


/tmp/ipykernel_40887/1248818640.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  height_predict_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
/tmp/ipykernel_40887/1248818640.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  height_predict_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)


In [24]:
height_predict_train.isna().sum()

subject_id       0
anchor_age       0
gender           0
weight           0
height        2072
dtype: int64

In [22]:
height_predict_train['weight'].describe()

count    10448.000000
mean        84.052687
std         24.774547
min         22.700000
25%         67.300000
50%         80.000000
75%         96.600000
max        296.800000
Name: weight, dtype: float64

In [21]:
height_predict_test.isna().sum()

subject_id      0
anchor_age      0
gender          0
weight          0
height        912
dtype: int64

In [23]:
height_predict_test['weight'].describe()

count    4481.000000
mean       83.833093
std        24.459891
min        21.100000
25%        67.700000
50%        80.000000
75%        96.000000
max       274.000000
Name: weight, dtype: float64

In [36]:
# 이제 키 통계량 보자
# train에 있는 환자들의 경우
height_predict_train['height'].describe()

count    8376.000000
mean      168.857825
std        13.087093
min         0.000000
25%       162.560000
50%       170.180000
75%       177.800000
max       444.500000
Name: height, dtype: float64

In [37]:
# test에 잇는 환자들의 경우
height_predict_test['height'].describe()

count    3569.000000
mean      168.675404
std        13.919066
min         0.000000
25%       162.560000
50%       170.180000
75%       177.800000
max       422.579800
Name: height, dtype: float64

In [39]:
# 너무 큰 키이거나 너무 작은 키가 있으면 이상치라고 판단하고 nan 값으로 먼저 처리해주는 과정

# 키는 cm임
# https://www.hani.co.kr/arti/international/international_general/1129377.html
# 세계에서 가장 작은 사람의 키 : 63㎝ -> 70cm 까지도 진짜 잘 없을 것 같아서 이상치 기준 70 이하로 하겠음
# 세계에서 가장 큰 사람의 키 : 251㎝
height_predict_train.loc[(height_predict_train['height'] <= 70) | (height_predict_train['height'] >= 251), 'height'] = np.nan
height_predict_test.loc[(height_predict_test['height'] <= 70) | (height_predict_test['height'] >= 251), 'height'] = np.nan

In [40]:
height_predict_train['height'].describe()

count    8350.000000
mean      169.184364
std        10.828546
min        71.120000
25%       162.560000
50%       170.180000
75%       177.800000
max       226.060000
Name: height, dtype: float64

In [41]:
height_predict_test['height'].describe()

count    3555.000000
mean      168.944335
std        10.558103
min       106.680000
25%       162.560000
50%       170.180000
75%       177.800000
max       200.660000
Name: height, dtype: float64

In [45]:
height_predict_train.isna().sum()

subject_id       0
anchor_age       0
gender           0
weight           0
height        2098
dtype: int64

In [47]:
height_predict_test.isna().sum()


subject_id      0
anchor_age      0
gender          0
weight          0
height        926
dtype: int64

#### 이제 진짜 키가 nan 인 사람의 키를 예측해보자 ㅇㅇ

In [57]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF를 계산하기 위해 독립 변수 준비
X_train = height_predict_train[['anchor_age', 'gender', 'weight']].dropna()  # 결측치가 없는 행만 선택
# VIF 계산
X_vif = sm.add_constant(X_train)  # 상수항 추가
vif_data = pd.DataFrame()
vif_data['feature'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
print("VIF 결과:")
vif_data
# vif값이 다 낮을걸로 보아 3개 변수간 다중공선성 문제는 걱정하지 않아도 될듯하다

VIF 결과:


,feature,VIF
0,const,38.089053
1,anchor_age,1.033856
2,gender,1.066157
3,weight,1.099750


In [58]:
from sklearn.linear_model import LinearRegression
train_non_null = height_predict_train.dropna(subset=['height']) # height 결측치가 없는 데이터만 사용
train_null = height_predict_train[height_predict_train['height'].isna()]  # height 결측치가 있는 데이터

X_train_non_null = train_non_null[['anchor_age', 'gender', 'weight']] #  # 결측치가 없는 데이터에서 독립 변수
y_train_non_null = train_non_null['height'] # 종속 변수는 height

# Linear Regression 모델을 사용해 결측치 예측 모델 학습
fill_model = LinearRegression()
fill_model.fit(X_train_non_null, y_train_non_null)

# train 데이터에 있는 환자들의 height 결측치 예측 및 채움
train_null['height'] = fill_model.predict(train_null[['anchor_age', 'gender', 'weight']])
X_train_filled = pd.concat([train_non_null, train_null])

### Test 데이터의 결측치 예측 및 채움 (Train에서 학습한 모델로)
X_test_null = height_predict_test[height_predict_test['height'].isna()]
X_test_non_null = height_predict_test.dropna(subset=['height'])

X_test_null['height'] = fill_model.predict(X_test_null[['anchor_age', 'gender', 'weight']])
X_test_filled = pd.concat([X_test_non_null, X_test_null])


# height_predict_test

/tmp/ipykernel_40887/2298332385.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_null['height'] = fill_model.predict(train_null[['anchor_age', 'gender', 'weight']])
/tmp/ipykernel_40887/2298332385.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_null['height'] = fill_model.predict(X_test_null[['anchor_age', 'gender', 'weight']])


In [65]:
print(X_train_filled['height'].describe())
X_train_filled

count    10448.000000
mean       168.926602
std         10.280996
min         71.120000
25%        160.576666
50%        170.180000
75%        176.530000
max        226.060000
Name: height, dtype: float64


,subject_id,anchor_age,gender,weight,height
0,10001884,68,1,65.0,157.480000
17,10002428,80,1,55.0,149.860000
42,10003400,72,1,99.5,167.640000
74,10003637,57,0,81.0,175.260000
81,10004401,82,0,77.5,170.180000
...,...,...,...,...,...
106694,19978239,82,1,101.0,162.140868
106699,19979275,63,0,78.1,174.017943
106817,19991743,80,1,53.0,157.566458
106833,19993155,62,1,100.0,163.092339


In [66]:
print(X_test_filled['height'].describe())
X_test_filled

count    4481.000000
mean      168.694692
std        10.019971
min       106.680000
25%       160.939092
50%       170.180000
75%       175.398928
max       200.660000
Name: height, dtype: float64


,subject_id,anchor_age,gender,weight,height
7,10004235,47,0,130.0,182.880000
12,10004606,64,1,57.1,162.560000
18,10004733,51,0,109.0,180.340000
27,10005606,38,0,95.0,177.800000
34,10005817,66,0,91.0,172.720000
...,...,...,...,...,...
45561,19918916,59,1,102.1,163.454402
45581,19920776,59,1,100.6,163.308174
45625,19941474,76,0,102.5,175.714769
45860,19967903,51,1,74.3,161.163886


In [93]:
train2 = pd.merge(train.drop(columns=['height']), X_train_filled[['subject_id', 'height']], on='subject_id', how='inner')
train2.head()
test2 = pd.merge(test.drop(columns=['height']), X_test_filled[['subject_id', 'height']], on='subject_id', how='inner')
test2.head()

,subject_id,hadm_id,chartdate,SBP_min,SBP_max,SBP_mean,DBP_min,DBP_max,DBP_mean,MAP_min,MAP_max,MAP_mean,HR_min,HR_max,HR_mean,SpO2_min,SpO2_max,SpO2_mean,Creatinine_mean,WBC_mean,Hemoglobin_mean,Platelet_mean,Potassium_mean,Chloride_mean,Glucose_mean,Sodium_mean,gender,anchor_age,race,medicine,weight,diagnose,AKI,height
0,10002155,23822395,2129-08-04,98.0,131.0,117.333333,46.0,69.0,60.250000,66.0,88.0,77.250000,68.0,83.0,74.500000,89.0,97.0,92.250000,0.9,5.5,12.5,185.0,4.5,106.0,95.0,139.0,1,80,WHITE,1,21.1,2,0,154.456669
1,10002155,23822395,2129-08-05,84.0,127.0,102.652174,42.0,65.0,52.818182,43.0,82.0,64.000000,65.0,93.0,76.458333,88.0,97.0,92.375000,1.2,6.5,12.1,165.0,4.7,106.0,107.0,140.0,1,80,WHITE,1,21.1,2,0,154.456669
2,10002155,23822395,2129-08-06,87.0,136.0,108.391304,36.0,75.0,50.304348,49.0,87.0,66.125000,61.0,96.0,81.875000,87.0,96.0,92.538462,1.2,5.3,10.4,158.0,4.6,106.0,106.0,141.0,1,80,WHITE,1,21.1,2,0,154.456669
3,10002155,23822395,2129-08-07,89.0,135.0,117.208333,43.0,67.0,57.083333,56.0,80.0,71.125000,75.0,98.0,82.240000,87.0,98.0,93.960000,1.1,5.4,10.8,146.0,4.5,106.0,116.0,138.0,1,80,WHITE,1,21.1,2,0,154.456669
4,10002155,23822395,2129-08-08,80.0,135.0,108.291667,38.0,82.0,53.956522,50.0,85.0,66.833333,67.0,96.0,81.434783,81.0,99.0,93.500000,1.1,3.8,9.9,139.0,4.2,106.0,102.0,141.0,1,80,WHITE,1,21.1,2,0,154.456669


In [95]:
res = train2.isna().sum()
print(res[res>0])
res = test2.isna().sum()
print(res[res>0])
# 이제 train test 둘다 nan값이 하나도 없다

Series([], dtype: int64)
Series([], dtype: int64)


In [96]:
# 체크포인트 check point
train2.to_csv('진짜_최종_nan값처리한train.csv',index = False)
test2.to_csv('진짜_최종_nan값처리한test.csv',index = False)

## AKI 예측 모델
- 보호변수 학습에 포함하지 않는 경우

In [118]:
import pandas as pd
train2 = pd.read_csv('진짜_최종_nan값처리한train.csv')
test2 = pd.read_csv('진짜_최종_nan값처리한test.csv') # 이거 다시 valid test 로 나눠야함
train2.columns

Index(['subject_id', 'hadm_id', 'chartdate', 'SBP_min', 'SBP_max', 'SBP_mean',
       'DBP_min', 'DBP_max', 'DBP_mean', 'MAP_min', 'MAP_max', 'MAP_mean',
       'HR_min', 'HR_max', 'HR_mean', 'SpO2_min', 'SpO2_max', 'SpO2_mean',
       'Creatinine_mean', 'WBC_mean', 'Hemoglobin_mean', 'Platelet_mean',
       'Potassium_mean', 'Chloride_mean', 'Glucose_mean', 'Sodium_mean',
       'gender', 'anchor_age', 'race', 'medicine', 'weight', 'diagnose', 'AKI',
       'height'],
      dtype='object')

In [119]:
# len(set(train2['hadm_id']))

In [120]:
# train2['count'] = 1
# train2.groupby(['hadm_id','race']).agg({'count':np.sum})

In [121]:
# 데이터에 1대1 매핑되는 칼럼들 merge해줄거임)
hadm = ['hadm_id']
onebyone = ['medicine','weight','diagnose','height']
hadm.extend(onebyone)
# 보호변수 (성별, 연령, 직업, 종교, 인종 등등)
protect_variable = ['gender','anchor_age','race']
hadm.extend(protect_variable)
d_onebyone_table_train = train2[hadm]
d_onebyone_table_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
d_onebyone_table_train

d_onebyone_table_test = test2[hadm]
d_onebyone_table_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
d_onebyone_table_test

/tmp/ipykernel_50319/3882763515.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_onebyone_table_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
/tmp/ipykernel_50319/3882763515.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_onebyone_table_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)


,hadm_id,medicine,weight,diagnose,height,gender,anchor_age,race
0,23822395,1,21.1,2,154.456669,1,80,WHITE
7,24181354,1,130.0,1,182.880000,0,47,BLACK/CAPE VERDEAN
12,29242151,1,57.1,1,162.560000,1,64,WHITE
18,27411876,1,109.0,0,180.340000,0,51,UNKNOWN
27,29646384,1,95.0,1,177.800000,0,38,WHITE
...,...,...,...,...,...,...,...,...
45953,29017569,1,85.5,2,165.100000,1,72,WHITE
45958,23560173,1,85.0,0,160.020000,1,52,WHITE
45963,28992962,1,110.0,1,152.400000,1,62,UNKNOWN
45970,25252564,1,113.4,0,182.880000,0,50,BLACK/AFRICAN AMERICAN


In [122]:
train2.drop(columns = onebyone,axis =1 , inplace = True)
test2.drop(columns = onebyone,axis =1 , inplace = True)

In [123]:
# creatinine_mean은 AKI target 칼럼 만들 떄 썻으니까 학습할 떄 변수로 넣어주면 안된다. 일종의 정답지 유출이라고 볼 수 있을듯
train2.drop(columns = ['Creatinine_mean'], axis = 1, inplace = True)
test2.drop(columns = ['Creatinine_mean'], axis = 1, inplace = True)

# 보호변수 (성별, 연령, 직업, 종교, 인종 등등)
protect_variable = ['gender','anchor_age','race']
# 보호변수도 학습에 사용안할거임
train2.drop(columns = protect_variable, axis = 1, inplace = True)
test2.drop(columns = protect_variable, axis = 1, inplace = True)


In [124]:
print(train2.columns, len(train2.columns))
print(test2.columns)

Index(['subject_id', 'hadm_id', 'chartdate', 'SBP_min', 'SBP_max', 'SBP_mean',
       'DBP_min', 'DBP_max', 'DBP_mean', 'MAP_min', 'MAP_max', 'MAP_mean',
       'HR_min', 'HR_max', 'HR_mean', 'SpO2_min', 'SpO2_max', 'SpO2_mean',
       'WBC_mean', 'Hemoglobin_mean', 'Platelet_mean', 'Potassium_mean',
       'Chloride_mean', 'Glucose_mean', 'Sodium_mean', 'AKI'],
      dtype='object') 26
Index(['subject_id', 'hadm_id', 'chartdate', 'SBP_min', 'SBP_max', 'SBP_mean',
       'DBP_min', 'DBP_max', 'DBP_mean', 'MAP_min', 'MAP_max', 'MAP_mean',
       'HR_min', 'HR_max', 'HR_mean', 'SpO2_min', 'SpO2_max', 'SpO2_mean',
       'WBC_mean', 'Hemoglobin_mean', 'Platelet_mean', 'Potassium_mean',
       'Chloride_mean', 'Glucose_mean', 'Sodium_mean', 'AKI'],
      dtype='object')


In [125]:
import pandas as pd
import numpy as np

# 슬라이딩 윈도우 생성 함수
def create_sliding_windows_as_df(data, time_steps=2):
    X, y = [], []
    columns = data.drop(columns=['AKI']).columns
    
    for i in range(len(data) - time_steps):
        # X: 현재 타임스텝에 해당하는 윈도우 데이터 (time_steps * 특성 수 크기)
        window_data = data.iloc[i:i + time_steps].drop(columns=['AKI']).values.flatten()
        X.append(window_data)
        
        # y: 현재 윈도우 끝난 후 48시간 이내의 AKI 여부
        y_values = data.iloc[i + time_steps:i + time_steps + 2]['AKI'].values
        y.append(1 if 1 in y_values else 0)
    
    # 컬럼 이름 만들기 (time_steps별로 컬럼에 번호 붙이기)
    flattened_columns = [f"{col}_t{t}" for t in range(1, time_steps + 1) for col in columns]
    
    # X를 데이터프레임으로 변환
    X_df = pd.DataFrame(X, columns=flattened_columns)
    y_df = pd.Series(y, name='AKI')
    
    return X_df, y_df

# 전체 데이터에 대해 슬라이딩 윈도우 적용
X_all, y_all = [], []
for subject_id, group in train2.groupby('hadm_id'):
    group = group.sort_values(by='chartdate')
    X, y = create_sliding_windows_as_df(group)
    if not X.empty:  # 비어 있지 않은 경우만 추가
        X_all.append(X)
        y_all.append(y)

# X_all, y_all을 하나의 데이터프레임으로 연결
X_all = pd.concat(X_all, ignore_index=True)  # shape: (샘플 수, time_steps * 특성 수)
y_all = pd.concat(y_all, ignore_index=True)  # shape: (샘플 수,)

# 최종 데이터프레임 형태 확인
print(X_all.shape)  # (샘플 수, time_steps * 특성 수)
print(y_all.shape)  # (샘플 수,)
# X_all과 y_all을 데이터프레임으로 결합
train2_final = pd.concat([X_all, y_all], axis=1)
train2_final.head()  # 최종 데이터프레임 확인
# (25412, 165)
# (25412,)

# 전체 데이터에 대해 슬라이딩 윈도우 적용
X_all, y_all = [], []
for subject_id, group in test2.groupby('hadm_id'):
    group = group.sort_values(by='chartdate')
    X, y = create_sliding_windows_as_df(group)
    if not X.empty:  # 비어 있지 않은 경우만 추가
        X_all.append(X)
        y_all.append(y)

# X_all, y_all을 하나의 데이터프레임으로 연결
X_all = pd.concat(X_all, ignore_index=True)  # shape: (샘플 수, time_steps * 특성 수)
y_all = pd.concat(y_all, ignore_index=True)  # shape: (샘플 수,)

# 최종 데이터프레임 형태 확인
print(X_all.shape)  # (샘플 수, time_steps * 특성 수)
print(y_all.shape)  # (샘플 수,)
# X_all과 y_all을 데이터프레임으로 결합
test2_final = pd.concat([X_all, y_all], axis=1)
test2_final.head()  # 최종 데이터프레임 확인
# (25412, 165)
# (25412,)

(83948, 50)
(83948,)
(36189, 50)
(36189,)


,subject_id_t1,hadm_id_t1,chartdate_t1,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,...,SpO2_max_t2,SpO2_mean_t2,WBC_mean_t2,Hemoglobin_mean_t2,Platelet_mean_t2,Potassium_mean_t2,Chloride_mean_t2,Glucose_mean_t2,Sodium_mean_t2,AKI
0,14577567,20001361,2143-05-04,102.0,125.0,116.333333,75.0,86.0,79.833333,83.0,...,99.0,97.826087,14.0,11.7,265.0,3.50,112.0,112.000000,148.00,1
1,14577567,20001361,2143-05-07,123.0,159.0,140.166667,66.0,82.0,73.833333,83.0,...,100.0,98.461538,14.8,10.9,239.0,3.80,114.0,100.000000,149.00,0
2,14577567,20001361,2143-05-08,120.0,170.0,149.892857,64.0,96.0,78.428571,82.0,...,98.0,95.791667,14.5,11.0,249.0,4.05,116.0,125.333333,149.75,0
3,12504470,20003425,2155-07-22,76.0,120.0,96.363636,43.0,66.0,52.000000,58.0,...,100.0,98.250000,10.0,7.6,211.0,5.00,101.0,162.000000,133.00,0
4,12504470,20003425,2155-07-23,68.0,129.0,90.305556,35.0,88.0,57.861111,48.0,...,100.0,99.160000,7.6,9.4,191.0,4.40,103.0,144.000000,134.00,0


In [126]:
train2_final = train2_final[train2_final.columns[~train2_final.columns.str.contains('subject_id')]]
train2_final = train2_final[train2_final.columns[~train2_final.columns.str.contains('chartdate')]]
train2_final.columns[train2_final.columns.str.contains('hadm_id')][1:]
train2_final.drop(columns=list(train2_final.columns[train2_final.columns.str.contains('hadm_id')][1:]),
                  axis = 1, inplace=True)
train2_final.columns = ['hadm_id', 'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'DBP_min_t1',
       'DBP_max_t1', 'DBP_mean_t1', 'MAP_min_t1', 'MAP_max_t1', 'MAP_mean_t1',
       'HR_min_t1', 'HR_max_t1', 'HR_mean_t1', 'SpO2_min_t1', 'SpO2_max_t1',
       'SpO2_mean_t1', 'WBC_mean_t1', 'Hemoglobin_mean_t1', 'Platelet_mean_t1',
       'Potassium_mean_t1', 'Chloride_mean_t1', 'Glucose_mean_t1',
       'Sodium_mean_t1', 'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2',
       'DBP_min_t2', 'DBP_max_t2', 'DBP_mean_t2', 'MAP_min_t2', 'MAP_max_t2',
       'MAP_mean_t2', 'HR_min_t2', 'HR_max_t2', 'HR_mean_t2', 'SpO2_min_t2',
       'SpO2_max_t2', 'SpO2_mean_t2', 'WBC_mean_t2', 'Hemoglobin_mean_t2',
       'Platelet_mean_t2', 'Potassium_mean_t2', 'Chloride_mean_t2',
       'Glucose_mean_t2', 'Sodium_mean_t2', 'AKI']
train2_final

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,SpO2_max_t2,SpO2_mean_t2,WBC_mean_t2,Hemoglobin_mean_t2,Platelet_mean_t2,Potassium_mean_t2,Chloride_mean_t2,Glucose_mean_t2,Sodium_mean_t2,AKI
0,20002506,98.0,114.0,107.055556,53.0,92.0,64.500000,72.0,99.0,77.055556,...,100.0,98.200000,8.00,13.00,174.5,3.950000,105.000000,96.5,143.333333,0
1,20002506,105.0,136.0,118.400000,61.0,80.0,70.200000,74.0,92.0,83.333333,...,100.0,98.750000,9.20,12.75,179.5,3.900000,101.000000,114.0,140.000000,0
2,20002506,110.0,133.0,123.562500,65.0,86.0,73.312500,80.0,98.0,87.250000,...,100.0,98.684211,8.90,12.10,177.0,3.700000,95.000000,131.0,136.000000,0
3,20002970,95.0,170.0,130.863636,53.0,115.0,79.318182,64.0,128.0,94.772727,...,97.0,94.764706,7.30,14.90,303.0,3.500000,97.000000,164.0,135.000000,0
4,20002970,110.0,162.0,133.411765,63.0,105.0,82.294118,81.0,114.0,95.875000,...,96.0,93.857143,15.60,14.10,282.0,3.800000,98.000000,137.0,134.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83943,29999098,137.0,218.0,178.956522,55.0,94.0,69.260870,82.0,128.0,103.521739,...,100.0,96.448276,9.00,10.60,53.0,3.933333,109.666667,170.0,141.333333,0
83944,29999098,149.0,223.0,190.206897,57.0,91.0,75.827586,84.0,137.0,112.931034,...,98.0,96.448276,10.80,9.50,23.0,3.866667,107.666667,232.0,141.000000,0
83945,29999098,99.0,218.0,158.551724,47.0,92.0,64.793103,66.0,139.0,94.206897,...,100.0,96.875000,13.30,8.10,23.0,3.933333,105.333333,273.5,139.000000,1
83946,29999098,139.0,189.0,164.333333,53.0,81.0,68.666667,71.0,114.0,94.208333,...,100.0,96.666667,15.60,7.10,37.0,3.900000,106.500000,124.5,139.500000,1


In [127]:
train2_final = pd.merge(train2_final,d_onebyone_table_train,how='left',on='hadm_id')
# train2_final.drop(columns=['hadm_id'],axis = 1, inplace= True)
train2_final

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,Glucose_mean_t2,Sodium_mean_t2,AKI,medicine,weight,diagnose,height,gender,anchor_age,race
0,20002506,98.0,114.0,107.055556,53.0,92.0,64.500000,72.0,99.0,77.055556,...,96.5,143.333333,0,1,60.0,0,174.298920,0,24,UNKNOWN
1,20002506,105.0,136.0,118.400000,61.0,80.0,70.200000,74.0,92.0,83.333333,...,114.0,140.000000,0,1,60.0,0,174.298920,0,24,UNKNOWN
2,20002506,110.0,133.0,123.562500,65.0,86.0,73.312500,80.0,98.0,87.250000,...,131.0,136.000000,0,1,60.0,0,174.298920,0,24,UNKNOWN
3,20002970,95.0,170.0,130.863636,53.0,115.0,79.318182,64.0,128.0,94.772727,...,164.0,135.000000,0,1,57.9,0,158.306376,1,75,WHITE
4,20002970,110.0,162.0,133.411765,63.0,105.0,82.294118,81.0,114.0,95.875000,...,137.0,134.000000,0,1,57.9,0,158.306376,1,75,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83943,29999098,137.0,218.0,178.956522,55.0,94.0,69.260870,82.0,128.0,103.521739,...,170.0,141.333333,0,1,91.6,0,167.640000,0,76,WHITE
83944,29999098,149.0,223.0,190.206897,57.0,91.0,75.827586,84.0,137.0,112.931034,...,232.0,141.000000,0,1,91.6,0,167.640000,0,76,WHITE
83945,29999098,99.0,218.0,158.551724,47.0,92.0,64.793103,66.0,139.0,94.206897,...,273.5,139.000000,1,1,91.6,0,167.640000,0,76,WHITE
83946,29999098,139.0,189.0,164.333333,53.0,81.0,68.666667,71.0,114.0,94.208333,...,124.5,139.500000,1,1,91.6,0,167.640000,0,76,WHITE


In [128]:
test2_final = test2_final[test2_final.columns[~test2_final.columns.str.contains('subject_id')]]
test2_final = test2_final[test2_final.columns[~test2_final.columns.str.contains('chartdate')]]
test2_final.columns[test2_final.columns.str.contains('hadm_id')][1:]
test2_final.drop(columns=list(test2_final.columns[test2_final.columns.str.contains('hadm_id')][1:]),
                  axis = 1, inplace=True)
test2_final.columns = ['hadm_id', 'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'DBP_min_t1',
       'DBP_max_t1', 'DBP_mean_t1', 'MAP_min_t1', 'MAP_max_t1', 'MAP_mean_t1',
       'HR_min_t1', 'HR_max_t1', 'HR_mean_t1', 'SpO2_min_t1', 'SpO2_max_t1',
       'SpO2_mean_t1', 'WBC_mean_t1', 'Hemoglobin_mean_t1', 'Platelet_mean_t1',
       'Potassium_mean_t1', 'Chloride_mean_t1', 'Glucose_mean_t1',
       'Sodium_mean_t1', 'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2',
       'DBP_min_t2', 'DBP_max_t2', 'DBP_mean_t2', 'MAP_min_t2', 'MAP_max_t2',
       'MAP_mean_t2', 'HR_min_t2', 'HR_max_t2', 'HR_mean_t2', 'SpO2_min_t2',
       'SpO2_max_t2', 'SpO2_mean_t2', 'WBC_mean_t2', 'Hemoglobin_mean_t2',
       'Platelet_mean_t2', 'Potassium_mean_t2', 'Chloride_mean_t2',
       'Glucose_mean_t2', 'Sodium_mean_t2', 'AKI']
test2_final

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,SpO2_max_t2,SpO2_mean_t2,WBC_mean_t2,Hemoglobin_mean_t2,Platelet_mean_t2,Potassium_mean_t2,Chloride_mean_t2,Glucose_mean_t2,Sodium_mean_t2,AKI
0,20001361,102.0,125.0,116.333333,75.0,86.0,79.833333,83.0,95.0,87.666667,...,99.0,97.826087,14.0,11.7,265.0,3.500000,112.000000,112.000000,148.000000,1
1,20001361,123.0,159.0,140.166667,66.0,82.0,73.833333,83.0,104.0,91.166667,...,100.0,98.461538,14.8,10.9,239.0,3.800000,114.000000,100.000000,149.000000,0
2,20001361,120.0,170.0,149.892857,64.0,96.0,78.428571,82.0,118.0,98.615385,...,98.0,95.791667,14.5,11.0,249.0,4.050000,116.000000,125.333333,149.750000,0
3,20003425,76.0,120.0,96.363636,43.0,66.0,52.000000,58.0,84.0,66.909091,...,100.0,98.250000,10.0,7.6,211.0,5.000000,101.000000,162.000000,133.000000,0
4,20003425,68.0,129.0,90.305556,35.0,88.0,57.861111,48.0,97.0,69.055556,...,100.0,99.160000,7.6,9.4,191.0,4.400000,103.000000,144.000000,134.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36184,29996046,93.0,121.0,109.125000,47.0,69.0,58.583333,59.0,79.0,70.083333,...,100.0,99.791667,5.5,8.3,364.0,5.050000,97.000000,97.000000,133.000000,0
36185,29996046,91.0,142.0,116.041667,48.0,74.0,61.208333,59.0,87.0,74.000000,...,100.0,99.750000,5.9,7.8,380.0,4.500000,100.000000,82.000000,134.000000,0
36186,29998464,77.0,157.0,108.269231,41.0,137.0,63.538462,52.0,144.0,77.384615,...,98.0,95.500000,30.5,11.3,300.0,4.633333,91.666667,376.800000,129.333333,0
36187,29998464,90.0,157.0,111.791667,45.0,73.0,54.875000,59.0,96.0,71.958333,...,97.0,95.000000,21.9,10.2,250.0,4.150000,93.250000,289.000000,135.500000,0


In [129]:
test2_final = pd.merge(test2_final,d_onebyone_table_test,how='left',on='hadm_id')
# test2_final.drop(columns=['hadm_id'],axis = 1, inplace= True)
test2_final

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,Glucose_mean_t2,Sodium_mean_t2,AKI,medicine,weight,diagnose,height,gender,anchor_age,race
0,20001361,102.0,125.0,116.333333,75.0,86.0,79.833333,83.0,95.0,87.666667,...,112.000000,148.000000,1,1,92.0,1,182.880000,0,30,WHITE
1,20001361,123.0,159.0,140.166667,66.0,82.0,73.833333,83.0,104.0,91.166667,...,100.000000,149.000000,0,1,92.0,1,182.880000,0,30,WHITE
2,20001361,120.0,170.0,149.892857,64.0,96.0,78.428571,82.0,118.0,98.615385,...,125.333333,149.750000,0,1,92.0,1,182.880000,0,30,WHITE
3,20003425,76.0,120.0,96.363636,43.0,66.0,52.000000,58.0,84.0,66.909091,...,162.000000,133.000000,0,1,86.0,0,174.106257,0,76,WHITE
4,20003425,68.0,129.0,90.305556,35.0,88.0,57.861111,48.0,97.0,69.055556,...,144.000000,134.000000,0,1,86.0,0,174.106257,0,76,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36184,29996046,93.0,121.0,109.125000,47.0,69.0,58.583333,59.0,79.0,70.083333,...,97.000000,133.000000,0,1,75.1,0,165.100000,1,70,BLACK/CAPE VERDEAN
36185,29996046,91.0,142.0,116.041667,48.0,74.0,61.208333,59.0,87.0,74.000000,...,82.000000,134.000000,0,1,75.1,0,165.100000,1,70,BLACK/CAPE VERDEAN
36186,29998464,77.0,157.0,108.269231,41.0,137.0,63.538462,52.0,144.0,77.384615,...,376.800000,129.333333,0,1,80.0,3,175.260000,0,78,ASIAN
36187,29998464,90.0,157.0,111.791667,45.0,73.0,54.875000,59.0,96.0,71.958333,...,289.000000,135.500000,0,1,80.0,3,175.260000,0,78,ASIAN


- standard 스케일링

In [130]:
num_col_to_scale = ['weight', 'height'] + [col for col in train2_final.columns if 't1' in col or 't2' in col]
cat_col = ['medicine','diagnose']
target_col = ['AKI']

from sklearn.preprocessing import StandardScaler

# 스케일러 인스턴스 생성
scaler = StandardScaler()
train2_final[num_col_to_scale] = scaler.fit_transform(train2_final[num_col_to_scale])

# 테스트 데이터 스케일링
# 주의: 테스트 데이터는 'fit'하지 않고 'transform'만 실행
test2_final[num_col_to_scale] = scaler.transform(test2_final[num_col_to_scale])

- 범주형 변수 인코딩
    - medicine 칼럼 : 0,1로 이루어져 있음 -> 따로 인코딩 하지 않는다
    - diagnose 칼럼 : 0,1,2,3으로 이루어져 있음 -> 원핫인코딩 : 범주가 순서를 가지는 변수가 아니라서 원핫인코딩 선택

In [131]:
set(train2_final[cat_col[0]]) # {0, 1}
set(train2_final[cat_col[1]]) # {0, 1, 2, 3}


{0, 1, 2, 3}

In [132]:
print(train2_final.shape)
train2_final.head()

(83948, 53)


,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,Glucose_mean_t2,Sodium_mean_t2,AKI,medicine,weight,diagnose,height,gender,anchor_age,race
0,20002506,0.134587,-1.414413,-0.751471,0.403901,0.347364,0.121452,0.747780,-0.160899,-0.214469,...,-0.902148,0.713402,0,1,-0.971353,0,0.503621,0,24,UNKNOWN
1,20002506,0.538195,-0.490483,-0.113251,1.146699,-0.251102,0.614415,0.908816,-0.517510,0.317285,...,-0.529653,0.087905,0,1,-0.971353,0,0.503621,0,24,UNKNOWN
2,20002506,0.826487,-0.616474,0.177183,1.518098,0.048131,0.883599,1.391925,-0.211843,0.649043,...,-0.167801,-0.662691,0,1,-0.971353,0,0.503621,0,24,UNKNOWN
3,20002970,-0.038388,0.937408,0.587934,0.403901,1.494425,1.402999,0.103635,1.316494,1.286249,...,0.534617,-0.850340,0,1,-1.053834,0,-1.036481,1,75,WHITE
4,20002970,0.826487,0.601433,0.731287,1.332398,0.995703,1.660372,1.472443,0.603270,1.379616,...,-0.040089,-1.037989,0,1,-1.053834,0,-1.036481,1,75,WHITE


In [133]:
print(test2_final.shape)
test2_final.head()

(36189, 53)


,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,Glucose_mean_t2,Sodium_mean_t2,AKI,medicine,weight,diagnose,height,gender,anchor_age,race
0,20001361,0.365220,-0.952448,-0.229518,2.446595,0.048131,1.447551,1.633479,-0.364677,0.684336,...,-0.572224,1.589098,1,1,0.285508,1,1.329989,0,30,WHITE
1,20001361,1.576045,0.475443,1.111307,1.610947,-0.151358,0.928643,1.633479,0.093824,0.980801,...,-0.827649,1.776747,0,1,0.285508,1,1.329989,0,30,WHITE
2,20001361,1.403070,0.937408,1.658487,1.425248,0.546853,1.326061,1.552961,0.807048,1.611738,...,-0.288419,1.917484,0,1,0.285508,1,1.329989,0,30,WHITE
3,20003425,-1.133896,-1.162432,-1.352981,-0.524596,-0.949313,-0.959607,-0.379473,-0.925067,-1.073916,...,0.492046,-1.225638,0,1,0.049846,0,0.485067,0,76,WHITE
4,20003425,-1.595162,-0.784461,-1.693799,-1.267394,0.147876,-0.452710,-1.184654,-0.262788,-0.892102,...,0.108909,-1.037989,0,1,0.049846,0,0.485067,0,76,WHITE


In [134]:
train_encoded = pd.get_dummies(train2_final, columns=['diagnose'], prefix='diag')
test_encoded = pd.get_dummies(test2_final, columns=['diagnose'], prefix='diag')
final_train_encoded, final_test_encoded = train_encoded.align(test_encoded, join='outer', axis=1, fill_value=0)
final_train_encoded[['diag_0',	'diag_1',	'diag_2',	'diag_3']] = final_train_encoded[['diag_0',	'diag_1',	'diag_2',	'diag_3']].astype(int)
final_test_encoded[['diag_0',	'diag_1',	'diag_2',	'diag_3']] = final_test_encoded[['diag_0',	'diag_1',	'diag_2',	'diag_3']].astype(int)

In [135]:
final_train_encoded.head()

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,medicine,weight,height,gender,anchor_age,race,diag_0,diag_1,diag_2,diag_3
0,20002506,0.134587,-1.414413,-0.751471,0.403901,0.347364,0.121452,0.747780,-0.160899,-0.214469,...,1,-0.971353,0.503621,0,24,UNKNOWN,1,0,0,0
1,20002506,0.538195,-0.490483,-0.113251,1.146699,-0.251102,0.614415,0.908816,-0.517510,0.317285,...,1,-0.971353,0.503621,0,24,UNKNOWN,1,0,0,0
2,20002506,0.826487,-0.616474,0.177183,1.518098,0.048131,0.883599,1.391925,-0.211843,0.649043,...,1,-0.971353,0.503621,0,24,UNKNOWN,1,0,0,0
3,20002970,-0.038388,0.937408,0.587934,0.403901,1.494425,1.402999,0.103635,1.316494,1.286249,...,1,-1.053834,-1.036481,1,75,WHITE,1,0,0,0
4,20002970,0.826487,0.601433,0.731287,1.332398,0.995703,1.660372,1.472443,0.603270,1.379616,...,1,-1.053834,-1.036481,1,75,WHITE,1,0,0,0


In [136]:
final_test_encoded.head()

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,medicine,weight,height,gender,anchor_age,race,diag_0,diag_1,diag_2,diag_3
0,20001361,0.365220,-0.952448,-0.229518,2.446595,0.048131,1.447551,1.633479,-0.364677,0.684336,...,1,0.285508,1.329989,0,30,WHITE,0,1,0,0
1,20001361,1.576045,0.475443,1.111307,1.610947,-0.151358,0.928643,1.633479,0.093824,0.980801,...,1,0.285508,1.329989,0,30,WHITE,0,1,0,0
2,20001361,1.403070,0.937408,1.658487,1.425248,0.546853,1.326061,1.552961,0.807048,1.611738,...,1,0.285508,1.329989,0,30,WHITE,0,1,0,0
3,20003425,-1.133896,-1.162432,-1.352981,-0.524596,-0.949313,-0.959607,-0.379473,-0.925067,-1.073916,...,1,0.049846,0.485067,0,76,WHITE,1,0,0,0
4,20003425,-1.595162,-0.784461,-1.693799,-1.267394,0.147876,-0.452710,-1.184654,-0.262788,-0.892102,...,1,0.049846,0.485067,0,76,WHITE,1,0,0,0


In [137]:
final_train_encoded.columns

Index(['hadm_id', 'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'DBP_min_t1',
       'DBP_max_t1', 'DBP_mean_t1', 'MAP_min_t1', 'MAP_max_t1', 'MAP_mean_t1',
       'HR_min_t1', 'HR_max_t1', 'HR_mean_t1', 'SpO2_min_t1', 'SpO2_max_t1',
       'SpO2_mean_t1', 'WBC_mean_t1', 'Hemoglobin_mean_t1', 'Platelet_mean_t1',
       'Potassium_mean_t1', 'Chloride_mean_t1', 'Glucose_mean_t1',
       'Sodium_mean_t1', 'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2',
       'DBP_min_t2', 'DBP_max_t2', 'DBP_mean_t2', 'MAP_min_t2', 'MAP_max_t2',
       'MAP_mean_t2', 'HR_min_t2', 'HR_max_t2', 'HR_mean_t2', 'SpO2_min_t2',
       'SpO2_max_t2', 'SpO2_mean_t2', 'WBC_mean_t2', 'Hemoglobin_mean_t2',
       'Platelet_mean_t2', 'Potassium_mean_t2', 'Chloride_mean_t2',
       'Glucose_mean_t2', 'Sodium_mean_t2', 'AKI', 'medicine', 'weight',
       'height', 'gender', 'anchor_age', 'race', 'diag_0', 'diag_1', 'diag_2',
       'diag_3'],
      dtype='object')

- 전처리 끝났고 진짜 모델

In [138]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.linear_model import LogisticRegression,RidgeClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import plot_importance

import catboost
from catboost import CatBoostClassifier

from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.model_selection import train_test_split as tts
# from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import log_loss

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

import os
import warnings
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

os.environ['KMP_DUPLICATE_LIB_OK']='True'
warnings.filterwarnings("ignore", category=FutureWarning)

In [139]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[0, 1])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[0, 1])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[0, 1])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도(precision 0.2 목표): {:.4f}".format(precision))
    print("재현율(recall 0.75 목표): {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [140]:
final_test_encoded

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,medicine,weight,height,gender,anchor_age,race,diag_0,diag_1,diag_2,diag_3
0,20001361,0.365220,-0.952448,-0.229518,2.446595,0.048131,1.447551,1.633479,-0.364677,0.684336,...,1,0.285508,1.329989,0,30,WHITE,0,1,0,0
1,20001361,1.576045,0.475443,1.111307,1.610947,-0.151358,0.928643,1.633479,0.093824,0.980801,...,1,0.285508,1.329989,0,30,WHITE,0,1,0,0
2,20001361,1.403070,0.937408,1.658487,1.425248,0.546853,1.326061,1.552961,0.807048,1.611738,...,1,0.285508,1.329989,0,30,WHITE,0,1,0,0
3,20003425,-1.133896,-1.162432,-1.352981,-0.524596,-0.949313,-0.959607,-0.379473,-0.925067,-1.073916,...,1,0.049846,0.485067,0,76,WHITE,1,0,0,0
4,20003425,-1.595162,-0.784461,-1.693799,-1.267394,0.147876,-0.452710,-1.184654,-0.262788,-0.892102,...,1,0.049846,0.485067,0,76,WHITE,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36184,29996046,-0.153705,-1.120435,-0.635047,-0.153197,-0.799696,-0.390249,-0.298955,-1.179790,-0.805045,...,1,-0.378272,-0.382247,1,70,BLACK/CAPE VERDEAN,1,0,0,0
36185,29996046,-0.269021,-0.238502,-0.245927,-0.060348,-0.550335,-0.163227,-0.298955,-0.772233,-0.473287,...,1,-0.378272,-0.382247,1,70,BLACK/CAPE VERDEAN,1,0,0,0
36186,29998464,-1.076237,0.391449,-0.683192,-0.710296,2.591613,0.038294,-0.862582,2.131607,-0.186596,...,1,-0.185815,0.596174,0,78,ASIAN,0,0,0,1
36187,29998464,-0.326679,0.391449,-0.485025,-0.338897,-0.600207,-0.710963,-0.298955,-0.313732,-0.646225,...,1,-0.185815,0.596174,0,78,ASIAN,0,0,0,1


In [141]:
print(hadm)
hadm.remove('height')
hadm.remove('weight')
hadm.remove('diagnose')
hadm.remove('medicine')
print(hadm)

['hadm_id', 'medicine', 'weight', 'diagnose', 'height', 'gender', 'anchor_age', 'race']
['hadm_id', 'gender', 'anchor_age', 'race']


In [ ]:
test_res=pd.read_csv('../juhye/dd4_test.csv')
test_subject_hadm = test_res[['subject_id','hadm_id']].drop_duplicates()

X_train = final_train_encoded[final_train_encoded.columns.drop('AKI')].drop(columns =hadm,axis = 1)
y_train = final_train_encoded['AKI']

# test를 test/valid로 나누기
final_test_valid_encoded = pd.merge(test_subject_hadm,final_test_encoded,on = 'hadm_id',how = 'inner')
from sklearn.model_selection import train_test_split
df=final_test_valid_encoded
unique_subject_ids = df['subject_id'].unique()

# 고유한 subject_id를 기준으로 train/test split
valid_ids, test_ids = train_test_split(unique_subject_ids, test_size=0.4, random_state=42)

# train과 test 데이터프레임 생성
valid_df = df[df['subject_id'].isin(valid_ids)]
test_df = df[df['subject_id'].isin(test_ids)]
print(len(valid_df))
print(len(test_df))

X_valid = valid_df[final_train_encoded.columns.drop('AKI')].drop(columns = hadm,axis = 1)
y_valid = valid_df['AKI']
X_test = test_df[final_train_encoded.columns.drop('AKI')].drop(columns =hadm,axis = 1)
y_test = test_df['AKI']

KeyboardInterrupt: 

In [143]:
X_test.shape

(14726, 51)

In [144]:
# 양성 클래스와 음성 클래스의 비율 계산
ratio =(len(y_train) - sum(y_train)) / sum(y_train)
ratio

3.86824402690791

In [34]:
import optuna
# from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Optuna를 위한 목적 함수
def objective(trial):
    # 탐색할 하이퍼파라미터 설정
    param = {
        'random_state': 42,
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', ratio-2, ratio+2),
        'n_estimators': trial.suggest_int('n_estimators', 300, 500),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 1e-1),
        'gamma': trial.suggest_loguniform('gamma', 1e-6, 1e-1),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12)
    }

    # 모델 생성 및 학습
    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)

    # 예측 및 평가
    y_pred = model.predict(X_valid)
#     score = f1_score(y_test, y_pred, average='binary')
    # score = precision_score(y_test, y_pred, average='binary')
    score = recall_score(y_valid, y_pred, average='binary')
    return score

# Optuna Study 생성 및 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=60)

# 최적화 결과 출력
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-09-25 20:50:59,094] A new study created in memory with name: no-name-ae6cb3e2-ef3e-46b0-9595-45a051549faa
[I 2024-09-25 20:51:14,742] Trial 0 finished with value: 0.33303431262614935 and parameters: {'scale_pos_weight': 3.512360123446058, 'n_estimators': 428, 'alpha': 1.2219205805680303e-05, 'gamma': 0.01513689296542449, 'learning_rate': 0.07736202721184104, 'max_depth': 10}. Best is trial 0 with value: 0.33303431262614935.
[I 2024-09-25 20:51:23,681] Trial 1 finished with value: 0.5487777528593855 and parameters: {'scale_pos_weight': 4.325262262847237, 'n_estimators': 484, 'alpha': 0.0077518902941562245, 'gamma': 4.818205606889329e-05, 'learning_rate': 0.2509306661354241, 'max_depth': 5}. Best is trial 1 with value: 0.5487777528593855.
[I 2024-09-25 20:51:24,967] Trial 2 finished with value: 0.4227405247813411 and parameters: {'scale_pos_weight': 1.9747439025766413, 'n_estimators': 494, 'alpha': 4.4831411513032876e-05, 'gamma': 0.0043434161420882866, 'learning_rate': 0.0166062

Number of finished trials: 60
Best trial: {'scale_pos_weight': 5.848421307243368, 'n_estimators': 476, 'alpha': 1.5047452166545056e-05, 'gamma': 1.5482527880396597e-06, 'learning_rate': 0.01276320292990235, 'max_depth': 3}


In [145]:
recall_85 = {'scale_pos_weight': 5.848421307243368, 'n_estimators': 476, 'alpha': 1.5047452166545056e-05, 'gamma': 1.5482527880396597e-06, 'learning_rate': 0.01276320292990235, 'max_depth': 3}


In [146]:
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=recall_85['scale_pos_weight'],
                              max_depth = recall_85['max_depth'],
                              n_estimators= recall_85['n_estimators'],
                              alpha = recall_85['alpha'],
                              gamma = recall_85['gamma'],
                              learning_rate = recall_85['learning_rate'],
                             ) 

model_xgb_pre.fit(X_train, y_train)


XGBClassifier(alpha=1.5047452166545056e-05, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1.5482527880396597e-06, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01276320292990235, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=476, n_jobs=None,
              num_parallel_tree=None, ...)

In [147]:
pred_xgb = model_xgb_pre.predict(X_valid)
get_clf_eval(y_valid, pred_xgb)
# 정밀도 0.2 목표
# 재현율 0.75 목표

오차행렬:
 [[8244 8760]
 [ 628 3831]]

정확도: 0.5626
정밀도(precision 0.2 목표): 0.3043
재현율(recall 0.75 목표): 0.8592
F1: 0.4494


In [148]:
# 최적의 하이퍼파라미터를 사용하여 모델 학습
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=recall_85['scale_pos_weight'],
                              max_depth = recall_85['max_depth'],
                              n_estimators= recall_85['n_estimators'],
                              alpha = recall_85['alpha'],
                              gamma = recall_85['gamma'],
                              learning_rate = recall_85['learning_rate'])
model_xgb_pre.fit(pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid]))

# 테스트 데이터셋으로 모델 성능 평가
y_pred = model_xgb_pre.predict(X_test)
get_clf_eval(y_test, y_pred)


오차행렬:
 [[5530 6219]
 [ 390 2587]]

정확도: 0.5512
정밀도(precision 0.2 목표): 0.2938
재현율(recall 0.75 목표): 0.8690
F1: 0.4391


In [149]:
print(test_df['anchor_age'].value_counts().sort_index())
print('\n')
# 연령을 연령대로 매핑하는 함수
def map_age_to_decade(age):
    return f"{(age // 10) * 10}대"

# 연령대 컬럼 추가
test_df['age_group'] = test_df['anchor_age'].apply(map_age_to_decade)
print(test_df['age_group'].value_counts().sort_index())
valid_df['age_group'] = valid_df['anchor_age'].apply(map_age_to_decade)


# age_group value_count관찰 결과 인원수 차이가 너무 남 -> 연령대 좀더 러프하게 잡아보자
# def classify_age_group(row):
#     if row in ['10대', '20대']:
#         return 'Young Adults'
#     elif row in ['30대', '40대', '50대', '60대']:
#         return 'Middle-aged Adults'
#     else:
#         return 'Older Adults'
def classify_age_group(age):
    if age <= 29:
        return 'Young Adults'  # 청년층
    elif 30 <= age <= 64:
        return 'Middle-aged Adults'  # 중장년층
    elif age >= 65:
        return 'Older Adults'  # 노년층
    return 'Unknown'  # 예외 처리


# 새로운 열 'age_group_big' 추가
test_df['age_group_big'] = test_df['anchor_age'].apply(classify_age_group)
valid_df['age_group_big'] =  valid_df['anchor_age'].apply(classify_age_group)

# 연령대 별 인원수 확인
age_group_counts = test_df['age_group_big'].value_counts().sort_index()
print(f'\n\n{age_group_counts}')

anchor_age
18      4
19     22
20     27
21     31
22     51
     ... 
86    154
87    161
88    143
89    107
91    279
Name: count, Length: 73, dtype: int64


age_group
10대      26
20대     438
30대     860
40대    1171
50대    2899
60대    3542
70대    3296
80대    2215
90대     279
Name: count, dtype: int64


age_group_big
Middle-aged Adults    6642
Older Adults          7620
Young Adults           464
Name: count, dtype: int64


/tmp/ipykernel_50319/3286625268.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['age_group'] = test_df['anchor_age'].apply(map_age_to_decade)
/tmp/ipykernel_50319/3286625268.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['age_group'] = valid_df['anchor_age'].apply(map_age_to_decade)
/tmp/ipykernel_50319/3286625268.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [150]:
set(test_df['race'])

{'AMERICAN INDIAN/ALASKA NATIVE',
 'ASIAN',
 'ASIAN - ASIAN INDIAN',
 'ASIAN - CHINESE',
 'ASIAN - SOUTH EAST ASIAN',
 'BLACK/AFRICAN',
 'BLACK/AFRICAN AMERICAN',
 'BLACK/CAPE VERDEAN',
 'BLACK/CARIBBEAN ISLAND',
 'HISPANIC OR LATINO',
 'HISPANIC/LATINO - CENTRAL AMERICAN',
 'HISPANIC/LATINO - COLUMBIAN',
 'HISPANIC/LATINO - CUBAN',
 'HISPANIC/LATINO - DOMINICAN',
 'HISPANIC/LATINO - GUATEMALAN',
 'HISPANIC/LATINO - HONDURAN',
 'HISPANIC/LATINO - PUERTO RICAN',
 'HISPANIC/LATINO - SALVADORAN',
 'MULTIPLE RACE/ETHNICITY',
 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
 'OTHER',
 'PATIENT DECLINED TO ANSWER',
 'PORTUGUESE',
 'SOUTH AMERICAN',
 'UNABLE TO OBTAIN',
 'UNKNOWN',
 'WHITE',
 'WHITE - BRAZILIAN',
 'WHITE - EASTERN EUROPEAN',
 'WHITE - OTHER EUROPEAN',
 'WHITE - RUSSIAN'}

In [151]:
race_set = {'AMERICAN INDIAN/ALASKA NATIVE',
 'ASIAN',
 'ASIAN - ASIAN INDIAN',
 'ASIAN - CHINESE',
 'ASIAN - SOUTH EAST ASIAN',
 
 'BLACK/AFRICAN',
 'BLACK/AFRICAN AMERICAN',
 'BLACK/CAPE VERDEAN',
 'BLACK/CARIBBEAN ISLAND',
 
 'HISPANIC OR LATINO',
 'HISPANIC/LATINO - CENTRAL AMERICAN',
 'HISPANIC/LATINO - COLUMBIAN',
 'HISPANIC/LATINO - CUBAN',
 'HISPANIC/LATINO - DOMINICAN',
 'HISPANIC/LATINO - GUATEMALAN',
 'HISPANIC/LATINO - HONDURAN',
 'HISPANIC/LATINO - PUERTO RICAN',
 'HISPANIC/LATINO - SALVADORAN',
 'MULTIPLE RACE/ETHNICITY',
 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
 'OTHER',
 'PATIENT DECLINED TO ANSWER',
 'PORTUGUESE',
 'SOUTH AMERICAN',
 'UNABLE TO OBTAIN',
 'UNKNOWN',
 
 'WHITE',
 'WHITE - BRAZILIAN',
 'WHITE - EASTERN EUROPEAN',
 'WHITE - OTHER EUROPEAN',
 'WHITE - RUSSIAN'}
# 인종을 더 큰 카테고리로 매핑하는 함수
def classify_race(race):
    if "BLACK" in race.upper():
        return 'Black'
    elif "WHITE" in race.upper():
        return 'White'
    elif "ASIAN" in race.upper():
        return 'Asian'
    else:
        return 'Other'



test_df['race_group'] = test_df['race'].apply(classify_race)
valid_df['race_group'] = valid_df['race'].apply(classify_race)

# print(final_test_encoded['race_group'].value_counts().sort_index())

y_pred = model_xgb_pre.predict(X_test)  # 이진 예측
y_proba = model_xgb_pre.predict_proba(X_test)[:, 1]  # 클래스 1의 확률 예측
y_test # 실제 레이블
len(y_pred)

print(protect_variable)
# s_f_age = test_dftest_df['age_group'] # 이게 아마 보호변수인듯?
s_f_age = test_df['age_group_big']
s_f_gender = test_df['gender']
s_f_race = test_df['race_group']
# s_f_race = test_df['race'] # race_group보다 더 결과 안좋음
print(len(s_f_age))
print('\n')
# 예측 결과와 실제 레이블, 보호변수를 데이터프레임으로 구성
data = {
    'y_pred': y_pred,
    'y_proba': y_proba,
    'y_true': y_test,
    'age_group': s_f_age,
    'gender': s_f_gender,
    'race_group': s_f_race
}
res = pd.DataFrame(data)

# Disparate Impact 계산 함수
def calculate_disparate_impact(df, attribute, positive_outcome=True):
    # 긍정적 결과를 받은 비율 계산
    if positive_outcome:
        favorable_rate = df.groupby(attribute)['y_pred'].mean()
    else:
        favorable_rate = 1 - df.groupby(attribute)['y_pred'].mean()

    # 보호된 속성에 따른 비율 계산
    di_values = favorable_rate / favorable_rate.max()
    return di_values

# Equality of Opportunity Difference 계산 함수
def calculate_eod(df, attribute, positive_class=1):
    # 실제 긍정 클래스인 경우에 한정
    positive_actual = df[df['y_true'] == positive_class]

    # 긍정적 결과를 받은 비율 계산
    opportunity_rate = positive_actual.groupby(attribute)['y_pred'].mean()
    
    # 최소 및 최대 그룹 간의 차이
    eod_value = opportunity_rate.max() - opportunity_rate.min()
    return eod_value

# 예시로 'race_group'에 대한 DI와 EOD 계산
di_race = calculate_disparate_impact(res, 'race_group')
eod_race = calculate_eod(res, 'race_group')

di_age = calculate_disparate_impact(res, 'age_group')
eod_age = calculate_eod(res, 'age_group')

di_gender = calculate_disparate_impact(res, 'gender')
eod_gender = calculate_eod(res, 'gender')


print("Disparate Impact(DI) by Race:")
print(di_race)
print("\n\nEquality of Opportunity Difference(EOD) by Race:")
print(eod_race)
print('\n\n\n\n-------------------------------------------------')
print("Disparate Impact(DI) by age:")
print(di_age)
print("\n\nEquality of Opportunity Difference(EOD) by age:")
print(eod_age)
print('\n\n\n\n-------------------------------------------------')
print("Disparate Impact(DI) by gender:")
print(di_gender)
print("\n\nEquality of Opportunity Difference(EOD) by gender:")
print(eod_gender)


['gender', 'anchor_age', 'race']
14726


Disparate Impact(DI) by Race:
race_group
Asian    0.751870
Black    1.000000
Other    0.732646
White    0.822872
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by Race:
0.13490370717606148




-------------------------------------------------
Disparate Impact(DI) by age:
age_group
Middle-aged Adults    0.86428
Older Adults          1.00000
Young Adults          0.58723
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by age:
0.08837062716671917




-------------------------------------------------
Disparate Impact(DI) by gender:
gender
0    1.000000
1    0.904287
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by gender:
0.0023931341805578654


/tmp/ipykernel_50319/614463811.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['race_group'] = test_df['race'].apply(classify_race)
/tmp/ipykernel_50319/614463811.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['race_group'] = valid_df['race'].apply(classify_race)


- DI, EOD 공정성 metrics 평가
    - Disparate Impact (DI): 보호 속성에 따른 그룹에서 긍정 예측 비율의 최소값과 최대값의 비율로 계산합니다. 값이 1에 가까울수록 공정하다고 판단됩니다.
    - Equal Opportunity Differences (EOD): 실제 긍정인 경우, 보호 속성에 따른 그룹 간의 예측 긍정 비율 차이를 계산합니다. 값이 0에 가까울수록 공정하다고 판단됩니다.

In [152]:
print(test_df['race_group'].value_counts().sort_index())


race_group
Asian     375
Black    1574
Other    3514
White    9263
Name: count, dtype: int64


#### 공정성 지표가 아쉬움 -> 모델 튜닝할 때 공정성 지표도 써보자

In [153]:
# protect_input_corr = final_train_encoded.copy()

# # 연령대 컬럼 추가
# protect_input_corr['age_group_big'] = protect_input_corr['anchor_age'].apply(classify_age_group)

# # 연령대 별 인원수 확인
# age_group_counts = protect_input_corr['age_group_big'].value_counts().sort_index()
# print(f'\n\n{age_group_counts}')

# # 인종
# protect_input_corr['race_group'] = protect_input_corr['race'].apply(map_race_to_category)
# print('\n\n')

# print(protect_input_corr['race_group'].value_counts().sort_index())
# print('\n\n')

# # 성별은 걍 둠
# print(protect_input_corr['gender'].value_counts().sort_index())

# protect_input_corr # X_train과 다 똑같은데, hadmid랑 보호변수 값들이 붙어있음


In [154]:
valid_df

,subject_id,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,...,gender,anchor_age,race,diag_0,diag_1,diag_2,diag_3,age_group,age_group_big,race_group
3,12504470,20003425,-1.133896,-1.162432,-1.352981,-0.524596,-0.949313,-0.959607,-0.379473,-0.925067,...,0,76,WHITE,1,0,0,0,70대,Older Adults,White
4,12504470,20003425,-1.595162,-0.784461,-1.693799,-1.267394,0.147876,-0.452710,-1.184654,-0.262788,...,0,76,WHITE,1,0,0,0,70대,Older Adults,White
5,12504470,20003425,-1.133896,-1.414413,-1.864012,0.311051,0.746342,-0.046328,-0.057401,0.246658,...,0,76,WHITE,1,0,0,0,70대,Older Adults,White
6,17002995,20006309,-0.384338,-0.784461,-0.498876,-0.153197,0.048131,0.538151,-0.621027,-0.568455,...,0,51,WHITE - BRAZILIAN,1,0,0,0,50대,Middle-aged Adults,White
7,17002995,20006309,-0.096046,-0.490483,-0.448844,-0.246047,-0.400719,0.352705,-0.540509,-0.568455,...,0,51,WHITE - BRAZILIAN,1,0,0,0,50대,Middle-aged Adults,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36181,17027670,29996046,-0.038388,-0.952448,-0.705370,0.218201,-0.650080,-0.372232,0.103635,-1.026956,...,1,70,BLACK/CAPE VERDEAN,1,0,0,0,70대,Older Adults,Black
36182,17027670,29996046,-0.038388,-1.246426,-0.925716,-0.431747,-0.699952,-0.613668,-0.459991,-1.077901,...,1,70,BLACK/CAPE VERDEAN,1,0,0,0,70대,Older Adults,Black
36183,17027670,29996046,-0.153705,-0.952448,-0.695994,-0.060348,-0.899441,-0.530787,-0.459991,-1.077901,...,1,70,BLACK/CAPE VERDEAN,1,0,0,0,70대,Older Adults,Black
36184,17027670,29996046,-0.153705,-1.120435,-0.635047,-0.153197,-0.799696,-0.390249,-0.298955,-1.179790,...,1,70,BLACK/CAPE VERDEAN,1,0,0,0,70대,Older Adults,Black


In [ ]:
import optuna
from sklearn.metrics import recall_score

# Optuna를 위한 목적 함수
def objective(trial):
    # 탐색할 하이퍼파라미터 설정
    param = {
        'random_state': 42,
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', 2.0, ratio2+1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-6, 0.1),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12)
    }

    # 모델 생성 및 학습
    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)

    # 예측 및 평가
    y_pred = model.predict(X_valid)
    score = recall_score(y_valid, y_pred, average='binary')
    
    # 공정성 지표 계산
    # s_f_age = valid_df['age_group'] # 이게 아마 보호변수인듯?
    s_f_age = valid_df['age_group_big']
    s_f_gender = valid_df['gender']
    s_f_race = valid_df['race_group']
    # s_f_race = valid_df['race'] # race_group보다 더 결과 안좋음
    
    data = {
        'y_pred': y_pred,
        'y_true': y_valid,
        'age_group': s_f_age,
        'gender': s_f_gender,
        'race_group': s_f_race
    }
    res = pd.DataFrame(data)
    
    di_race = calculate_disparate_impact(res, 'race_group').min()
    eod_race = calculate_eod(res, 'race_group')
    di_age = calculate_disparate_impact(res, 'age_group').min()
    eod_age = calculate_eod(res, 'age_group')
    di_gender = calculate_disparate_impact(res, 'gender').min()
    eod_gender = calculate_eod(res, 'gender')
    
    # 공정성 지표 수정
    fairness_score = (0.4*di_race + 0.4*di_age +0.2*(1-np.abs(eod_race)))
    # 성능과 공정성의 조합
    # final_score = 0.2*score + fairness_score
    
    return fairness_score

# Optuna Study 생성 및 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=60)

# 최적화 결과 출력
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [156]:
final_score_with7_18 = {'scale_pos_weight': 4.716804368935939, 'n_estimators': 300, 'alpha': 0.00031392827856145786, 'gamma': 8.111907936802415e-05, 'learning_rate': 0.1297207410667147, 'max_depth': 3}
final_score_with2_29 =  {'scale_pos_weight': 4.340219309281246, 'n_estimators': 378, 'alpha': 0.047950863512757895, 'gamma': 2.141564908013647e-05, 'learning_rate': 0.047803855243832634, 'max_depth': 3}
final_score_가중치mean = {'scale_pos_weight': 4.849359809711112, 'n_estimators': 480, 'alpha': 0.0039283315993259875, 'gamma': 3.881299402679123e-06, 'learning_rate': 0.01367345952228048, 'max_depth': 6}
공정성_지표만_64 = {'scale_pos_weight': 5.36163163596889, 'n_estimators': 407, 'alpha': 3.7520405955907765e-05, 'gamma': 0.0013308599385402516, 'learning_rate': 0.01533825502155045, 'max_depth': 4} 
공정성_지표만_68 = {'scale_pos_weight': 5.866671669789028, 'n_estimators': 134, 'alpha': 0.05778648477561766, 'gamma': 1.5482895049562144e-05, 'learning_rate': 0.011632062878753404, 'max_depth': 4} #fairness_score = (0.4*di_race + 0.4*di_age +0.2*(1-np.abs(eod_race)))
공정성_지표만_train_valid_test_72 = {'scale_pos_weight': 5.856378062989231, 'n_estimators': 103, 'alpha': 1.0265875675750308e-05, 'gamma': 1.785478510690727e-06, 'learning_rate': 0.014202588851798054, 'max_depth': 7}# fairness_score = (0.4*di_race + 0.4*di_age +0.2*(1-np.abs(eod_race)))

In [161]:
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=공정성_지표만_train_valid_test_72['scale_pos_weight'],
                              max_depth = 공정성_지표만_train_valid_test_72['max_depth'],
                              n_estimators= 공정성_지표만_train_valid_test_72['n_estimators'],
                              alpha = 공정성_지표만_train_valid_test_72['alpha'],
                              gamma = 공정성_지표만_train_valid_test_72['gamma'],
                              learning_rate = 공정성_지표만_train_valid_test_72['learning_rate'],
                             ) 

model_xgb_pre.fit(X_train, y_train)
pred_xgb = model_xgb_pre.predict(X_valid)
get_clf_eval(y_valid, pred_xgb)
# 정밀도 0.2 목표
# 재현율 0.75 목표

오차행렬:
 [[8142 8862]
 [ 634 3825]]

정확도: 0.5576
정밀도(precision 0.2 목표): 0.3015
재현율(recall 0.75 목표): 0.8578
F1: 0.4462


In [162]:
# 최적의 하이퍼파라미터를 사용하여 모델 학습
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=공정성_지표만_train_valid_test_72['scale_pos_weight'],
                              max_depth = 공정성_지표만_train_valid_test_72['max_depth'],
                              n_estimators= 공정성_지표만_train_valid_test_72['n_estimators'],
                              alpha = 공정성_지표만_train_valid_test_72['alpha'],
                              gamma = 공정성_지표만_train_valid_test_72['gamma'],
                              learning_rate = 공정성_지표만_train_valid_test_72['learning_rate'])
model_xgb_pre.fit(pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid]))

# 테스트 데이터셋으로 모델 성능 평가
y_pred = model_xgb_pre.predict(X_test)
get_clf_eval(y_test, y_pred)


오차행렬:
 [[5453 6296]
 [ 374 2603]]

정확도: 0.5471
정밀도(precision 0.2 목표): 0.2925
재현율(recall 0.75 목표): 0.8744
F1: 0.4384


In [163]:
y_pred = model_xgb_pre.predict(X_test)  # 이진 예측
y_proba = model_xgb_pre.predict_proba(X_test)[:, 1]  # 클래스 1의 확률 예측
y_test # 실제 레이블

data = {
    'y_pred': y_pred,
    'y_proba': y_proba,
    'y_true': y_test,
    'age_group': s_f_age,
    'gender': s_f_gender,
    'race_group': s_f_race
}
res = pd.DataFrame(data)

di_race = calculate_disparate_impact(res, 'race_group')
eod_race = calculate_eod(res, 'race_group')

di_age = calculate_disparate_impact(res, 'age_group')
eod_age = calculate_eod(res, 'age_group')

di_gender = calculate_disparate_impact(res, 'gender')
eod_gender = calculate_eod(res, 'gender')


print("Disparate Impact(DI) by Race:")
print(di_race.min())
print(di_race)
print("\n\nEquality of Opportunity Difference(EOD) by Race:")
print(eod_race)
print('\n\n\n\n-------------------------------------------------')
print("Disparate Impact(DI) by age:")
print(di_age.min())
print(di_age)
print("\n\nEquality of Opportunity Difference(EOD) by age:")
print(eod_age)
print('\n\n\n\n-------------------------------------------------')
print("Disparate Impact(DI) by gender:")
print(di_gender.min())
print(di_gender)
print("\n\nEquality of Opportunity Difference(EOD) by gender:")
print(eod_gender)

Disparate Impact(DI) by Race:
0.7369300437999554
race_group
Asian    0.777419
Black    1.000000
Other    0.736930
White    0.833954
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by Race:
0.12786046329934764




-------------------------------------------------
Disparate Impact(DI) by age:
0.5852158730379123
age_group
Middle-aged Adults    0.867305
Older Adults          1.000000
Young Adults          0.585216
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by age:
0.0967538606996533




-------------------------------------------------
Disparate Impact(DI) by gender:
0.8909437715983123
gender
0    1.000000
1    0.890944
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by gender:
0.018091332664694648


## 다른 기준 튜닝

In [85]:
import optuna
from sklearn.metrics import recall_score

# Optuna를 위한 목적 함수
def objective(trial):
    # 탐색할 하이퍼파라미터 설정
    param = {
        'random_state': 42,
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', 2.0, ratio2+1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'alpha': trial.suggest_loguniform('alpha', 1e-5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-6, 0.1),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12)
    }

    # 모델 생성 및 학습
    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)

    # 예측 및 평가
    y_pred = model.predict(X_valid)
    score = recall_score(y_valid, y_pred, average='binary')
    
    # 공정성 지표 계산
    # s_f_age = valid_df['age_group'] # 이게 아마 보호변수인듯?
    s_f_age = valid_df['age_group_big']
    s_f_gender = valid_df['gender']
    s_f_race = valid_df['race_group']
    # s_f_race = valid_df['race'] # race_group보다 더 결과 안좋음
    
    data = {
        'y_pred': y_pred,
        'y_true': y_valid,
        'age_group': s_f_age,
        'gender': s_f_gender,
        'race_group': s_f_race
    }
    res = pd.DataFrame(data)
    
    di_race = calculate_disparate_impact(res, 'race_group').min()
    eod_race = calculate_eod(res, 'race_group')
    di_age = calculate_disparate_impact(res, 'age_group').min()
    eod_age = calculate_eod(res, 'age_group')
    di_gender = calculate_disparate_impact(res, 'gender').min()
    eod_gender = calculate_eod(res, 'gender')
    
    # 공정성 지표 수정
    fairness_score = (0.4*di_race + 0.4*di_age +0.1*(1-np.abs(eod_race))+0.1*(1-np.abs(eod_age)))
    # 성능과 공정성의 조합
    # final_score = 0.2*score + fairness_score
    
    return fairness_score

# Optuna Study 생성 및 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=60)

# 최적화 결과 출력
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-09-25 22:25:37,759] A new study created in memory with name: no-name-f55b5eac-33db-4ec0-a85c-f20b2c998154
[I 2024-09-25 22:25:51,456] Trial 0 finished with value: 0.6530688268697689 and parameters: {'scale_pos_weight': 4.105283385161891, 'n_estimators': 532, 'alpha': 0.2647392468338174, 'gamma': 0.0010437667201931545, 'learning_rate': 0.016433709849784723, 'max_depth': 8}. Best is trial 0 with value: 0.6530688268697689.
[I 2024-09-25 22:26:00,813] Trial 1 finished with value: 0.6281337803955837 and parameters: {'scale_pos_weight': 2.0801699971167773, 'n_estimators': 479, 'alpha': 0.027746309241510492, 'gamma': 0.0008405005816204868, 'learning_rate': 0.016212252820938692, 'max_depth': 6}. Best is trial 0 with value: 0.6530688268697689.
[I 2024-09-25 22:26:22,245] Trial 2 finished with value: 0.621215150780443 and parameters: {'scale_pos_weight': 3.0003841864344922, 'n_estimators': 598, 'alpha': 0.00023209746659801637, 'gamma': 3.900058366694336e-06, 'learning_rate': 0.0185016066

Number of finished trials: 60
Best trial: {'scale_pos_weight': 5.377167403257188, 'n_estimators': 134, 'alpha': 0.0030035712162084613, 'gamma': 0.00013317330659632233, 'learning_rate': 0.013731851478651295, 'max_depth': 6}


In [164]:
공정성지표_71 = {'scale_pos_weight': 5.377167403257188, 'n_estimators': 134, 'alpha': 0.0030035712162084613, 'gamma': 0.00013317330659632233, 'learning_rate': 0.013731851478651295, 'max_depth': 6}
#    fairness_score = (0.4*di_race + 0.4*di_age +0.1*(1-np.abs(eod_race))+0.1*(1-np.abs(eod_age)))


In [165]:
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=공정성지표_71['scale_pos_weight'],
                              max_depth = 공정성지표_71['max_depth'],
                              n_estimators= 공정성지표_71['n_estimators'],
                              alpha = 공정성지표_71['alpha'],
                              gamma = 공정성지표_71['gamma'],
                              learning_rate = 공정성지표_71['learning_rate'],
                             ) 

model_xgb_pre.fit(X_train, y_train)
pred_xgb = model_xgb_pre.predict(X_valid)
get_clf_eval(y_valid, pred_xgb)
# 정밀도 0.2 목표
# 재현율 0.75 목표

오차행렬:
 [[8608 8396]
 [ 707 3752]]

정확도: 0.5759
정밀도(precision 0.2 목표): 0.3089
재현율(recall 0.75 목표): 0.8414
F1: 0.4519


In [166]:
# 최적의 하이퍼파라미터를 사용하여 모델 학습
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=공정성지표_71['scale_pos_weight'],
                              max_depth = 공정성지표_71['max_depth'],
                              n_estimators= 공정성지표_71['n_estimators'],
                              alpha = 공정성지표_71['alpha'],
                              gamma = 공정성지표_71['gamma'],
                              learning_rate = 공정성지표_71['learning_rate'])
model_xgb_pre.fit(pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid]))

# 테스트 데이터셋으로 모델 성능 평가
y_pred = model_xgb_pre.predict(X_test)
get_clf_eval(y_test, y_pred)


오차행렬:
 [[5764 5985]
 [ 424 2553]]

정확도: 0.5648
정밀도(precision 0.2 목표): 0.2990
재현율(recall 0.75 목표): 0.8576
F1: 0.4434


In [167]:
y_pred = model_xgb_pre.predict(X_test)  # 이진 예측
y_proba = model_xgb_pre.predict_proba(X_test)[:, 1]  # 클래스 1의 확률 예측
y_test # 실제 레이블

data = {
    'y_pred': y_pred,
    'y_proba': y_proba,
    'y_true': y_test,
    'age_group': s_f_age,
    'gender': s_f_gender,
    'race_group': s_f_race
}
res = pd.DataFrame(data)

di_race = calculate_disparate_impact(res, 'race_group')
eod_race = calculate_eod(res, 'race_group')

di_age = calculate_disparate_impact(res, 'age_group')
eod_age = calculate_eod(res, 'age_group')

di_gender = calculate_disparate_impact(res, 'gender')
eod_gender = calculate_eod(res, 'gender')


print("Disparate Impact(DI) by Race:")
print(di_race.min())
print(di_race)
print("\n\nEquality of Opportunity Difference(EOD) by Race:")
print(eod_race)
print('\n\n\n\n-------------------------------------------------')
print("Disparate Impact(DI) by age:")
print(di_age.min())
print(di_age)
print("\n\nEquality of Opportunity Difference(EOD) by age:")
print(eod_age)
print('\n\n\n\n-------------------------------------------------')
print("Disparate Impact(DI) by gender:")
print(di_gender.min())
print(di_gender)
print("\n\nEquality of Opportunity Difference(EOD) by gender:")
print(eod_gender)

Disparate Impact(DI) by Race:
0.7218521469676848
race_group
Asian    0.749924
Black    1.000000
Other    0.721852
White    0.815481
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by Race:
0.14296261572717683




-------------------------------------------------
Disparate Impact(DI) by age:
0.590932044438032
age_group
Middle-aged Adults    0.860674
Older Adults          1.000000
Young Adults          0.590932
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by age:
0.10052526525895578




-------------------------------------------------
Disparate Impact(DI) by gender:
0.8929800763078211
gender
0    1.00000
1    0.89298
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by gender:
0.015006284357662492


# 최종 모델 -> 가장 처음에 recall 기준으로 튜닝했던 모델 파라미터를 사용하겠다

In [91]:
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=recall_85['scale_pos_weight'],
                              max_depth = recall_85['max_depth'],
                              n_estimators= recall_85['n_estimators'],
                              alpha = recall_85['alpha'],
                              gamma = recall_85['gamma'],
                              learning_rate = recall_85['learning_rate'],
                             ) 

model_xgb_pre.fit(X_train, y_train)
pred_xgb = model_xgb_pre.predict(X_valid)
get_clf_eval(y_valid, pred_xgb)
# 정밀도 0.2 목표
# 재현율 0.75 목표

오차행렬:
 [[8244 8760]
 [ 628 3831]]

정확도: 0.5626
정밀도(precision 0.2 목표): 0.3043
재현율(recall 0.75 목표): 0.8592
F1: 0.4494


In [168]:
# 최적의 하이퍼파라미터를 사용하여 모델 학습
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=recall_85['scale_pos_weight'],
                              max_depth = recall_85['max_depth'],
                              n_estimators= recall_85['n_estimators'],
                              alpha = recall_85['alpha'],
                              gamma = recall_85['gamma'],
                              learning_rate = recall_85['learning_rate'])
model_xgb_pre.fit(pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid]))

# 테스트 데이터셋으로 모델 성능 평가
y_pred = model_xgb_pre.predict(X_test)
get_clf_eval(y_test, y_pred)


오차행렬:
 [[5530 6219]
 [ 390 2587]]

정확도: 0.5512
정밀도(precision 0.2 목표): 0.2938
재현율(recall 0.75 목표): 0.8690
F1: 0.4391


In [93]:
y_pred = model_xgb_pre.predict(X_test)  # 이진 예측
y_proba = model_xgb_pre.predict_proba(X_test)[:, 1]  # 클래스 1의 확률 예측
y_test # 실제 레이블

data = {
    'y_pred': y_pred,
    'y_proba': y_proba,
    'y_true': y_test,
    'age_group': s_f_age,
    'gender': s_f_gender,
    'race_group': s_f_race
}
res = pd.DataFrame(data)

di_race = calculate_disparate_impact(res, 'race_group')
eod_race = calculate_eod(res, 'race_group')

di_age = calculate_disparate_impact(res, 'age_group')
eod_age = calculate_eod(res, 'age_group')

di_gender = calculate_disparate_impact(res, 'gender')
eod_gender = calculate_eod(res, 'gender')


print("Disparate Impact(DI) by Race:")
print(di_race.min())
print(di_race)
print("\n\nEquality of Opportunity Difference(EOD) by Race:")
print(eod_race)
print('\n\n\n\n-------------------------------------------------')
print("Disparate Impact(DI) by age:")
print(di_age.min())
print(di_age)
print("\n\nEquality of Opportunity Difference(EOD) by age:")
print(eod_age)
print('\n\n\n\n-------------------------------------------------')
print("Disparate Impact(DI) by gender:")
print(di_gender.min())
print(di_gender)
print("\n\nEquality of Opportunity Difference(EOD) by gender:")
print(eod_gender)

Disparate Impact(DI) by Race:
0.7326455668011185
race_group
Asian    0.751870
Black    1.000000
Other    0.732646
White    0.822872
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by Race:
0.13490370717606148




-------------------------------------------------
Disparate Impact(DI) by age:
0.5872297496111757
age_group
Middle-aged Adults    0.86428
Older Adults          1.00000
Young Adults          0.58723
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by age:
0.08837062716671917




-------------------------------------------------
Disparate Impact(DI) by gender:
0.9042874292023089
gender
0    1.000000
1    0.904287
Name: y_pred, dtype: float64


Equality of Opportunity Difference(EOD) by gender:
0.0023931341805578654


### case 6개 평가

In [169]:
import pandas as pd
train2 = pd.read_csv('/home/nhkim/med_datacon/knh_code/진짜_최종_nan값처리한train.csv')
test2 = pd.read_csv('/home/nhkim/med_datacon/knh_code/진짜_최종_nan값처리한test.csv')
train2.columns

Index(['subject_id', 'hadm_id', 'chartdate', 'SBP_min', 'SBP_max', 'SBP_mean',
       'DBP_min', 'DBP_max', 'DBP_mean', 'MAP_min', 'MAP_max', 'MAP_mean',
       'HR_min', 'HR_max', 'HR_mean', 'SpO2_min', 'SpO2_max', 'SpO2_mean',
       'Creatinine_mean', 'WBC_mean', 'Hemoglobin_mean', 'Platelet_mean',
       'Potassium_mean', 'Chloride_mean', 'Glucose_mean', 'Sodium_mean',
       'gender', 'anchor_age', 'race', 'medicine', 'weight', 'diagnose', 'AKI',
       'height'],
      dtype='object')

In [170]:
# 데이터에 1대1 매핑되는 칼럼들 merge해줄거임)
hadm = ['hadm_id']
onebyone = ['medicine','weight','diagnose','height']
hadm.extend(onebyone)
# 보호변수 (성별, 연령, 직업, 종교, 인종 등등)
protect_variable = ['gender','anchor_age','race']
hadm.extend(protect_variable)
d_onebyone_table_train = train2[hadm]
d_onebyone_table_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
d_onebyone_table_train

d_onebyone_table_test = test2[hadm]
d_onebyone_table_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
d_onebyone_table_test

/tmp/ipykernel_50319/3882763515.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_onebyone_table_train.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
/tmp/ipykernel_50319/3882763515.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_onebyone_table_test.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)


,hadm_id,medicine,weight,diagnose,height,gender,anchor_age,race
0,23822395,1,21.1,2,154.456669,1,80,WHITE
7,24181354,1,130.0,1,182.880000,0,47,BLACK/CAPE VERDEAN
12,29242151,1,57.1,1,162.560000,1,64,WHITE
18,27411876,1,109.0,0,180.340000,0,51,UNKNOWN
27,29646384,1,95.0,1,177.800000,0,38,WHITE
...,...,...,...,...,...,...,...,...
45953,29017569,1,85.5,2,165.100000,1,72,WHITE
45958,23560173,1,85.0,0,160.020000,1,52,WHITE
45963,28992962,1,110.0,1,152.400000,1,62,UNKNOWN
45970,25252564,1,113.4,0,182.880000,0,50,BLACK/AFRICAN AMERICAN


In [171]:
train2.drop(columns = onebyone,axis =1 , inplace = True)
test2.drop(columns = onebyone,axis =1 , inplace = True)

In [172]:
# creatinine_mean은 AKI target 칼럼 만들 떄 썻으니까 학습할 떄 변수로 넣어주면 안된다. 일종의 정답지 유출이라고 볼 수 있을듯
train2.drop(columns = ['Creatinine_mean'], axis = 1, inplace = True)
test2.drop(columns = ['Creatinine_mean'], axis = 1, inplace = True)

# 보호변수 (성별, 연령, 직업, 종교, 인종 등등)
protect_variable = ['gender','anchor_age','race']
# 보호변수도 학습에 사용안할거임
train2.drop(columns = protect_variable, axis = 1, inplace = True)
test2.drop(columns = protect_variable, axis = 1, inplace = True)
print(train2.columns, len(train2.columns))
print(test2.columns)

Index(['subject_id', 'hadm_id', 'chartdate', 'SBP_min', 'SBP_max', 'SBP_mean',
       'DBP_min', 'DBP_max', 'DBP_mean', 'MAP_min', 'MAP_max', 'MAP_mean',
       'HR_min', 'HR_max', 'HR_mean', 'SpO2_min', 'SpO2_max', 'SpO2_mean',
       'WBC_mean', 'Hemoglobin_mean', 'Platelet_mean', 'Potassium_mean',
       'Chloride_mean', 'Glucose_mean', 'Sodium_mean', 'AKI'],
      dtype='object') 26
Index(['subject_id', 'hadm_id', 'chartdate', 'SBP_min', 'SBP_max', 'SBP_mean',
       'DBP_min', 'DBP_max', 'DBP_mean', 'MAP_min', 'MAP_max', 'MAP_mean',
       'HR_min', 'HR_max', 'HR_mean', 'SpO2_min', 'SpO2_max', 'SpO2_mean',
       'WBC_mean', 'Hemoglobin_mean', 'Platelet_mean', 'Potassium_mean',
       'Chloride_mean', 'Glucose_mean', 'Sodium_mean', 'AKI'],
      dtype='object')


In [173]:
test2

,subject_id,hadm_id,chartdate,SBP_min,SBP_max,SBP_mean,DBP_min,DBP_max,DBP_mean,MAP_min,...,SpO2_max,SpO2_mean,WBC_mean,Hemoglobin_mean,Platelet_mean,Potassium_mean,Chloride_mean,Glucose_mean,Sodium_mean,AKI
0,10002155,23822395,2129-08-04,98.0,131.0,117.333333,46.0,69.0,60.250000,66.0,...,97.0,92.250000,5.5,12.5,185.0,4.50,106.0,95.0,139.0,0
1,10002155,23822395,2129-08-05,84.0,127.0,102.652174,42.0,65.0,52.818182,43.0,...,97.0,92.375000,6.5,12.1,165.0,4.70,106.0,107.0,140.0,0
2,10002155,23822395,2129-08-06,87.0,136.0,108.391304,36.0,75.0,50.304348,49.0,...,96.0,92.538462,5.3,10.4,158.0,4.60,106.0,106.0,141.0,0
3,10002155,23822395,2129-08-07,89.0,135.0,117.208333,43.0,67.0,57.083333,56.0,...,98.0,93.960000,5.4,10.8,146.0,4.50,106.0,116.0,138.0,0
4,10002155,23822395,2129-08-08,80.0,135.0,108.291667,38.0,82.0,53.956522,50.0,...,99.0,93.500000,3.8,9.9,139.0,4.20,106.0,102.0,141.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46002,19999297,21439025,2162-08-19,92.0,133.0,109.708333,50.0,84.0,67.333333,63.0,...,98.0,94.791667,8.1,9.3,55.5,3.30,111.5,78.5,147.0,1
46003,19999297,21439025,2162-08-20,102.0,150.0,124.521739,59.0,92.0,78.434783,75.0,...,99.0,96.285714,6.8,9.1,59.0,3.45,110.5,92.0,147.5,1
46004,19999297,21439025,2162-08-21,95.0,159.0,126.130435,63.0,100.0,77.173913,74.0,...,100.0,99.478261,6.9,7.8,56.0,3.35,109.5,109.0,147.0,1
46005,19999297,21439025,2162-08-22,102.0,138.0,122.833333,56.0,88.0,76.916667,69.0,...,100.0,97.208333,6.4,8.0,51.0,3.25,108.5,86.0,145.0,1


In [174]:
filtered_df = test2.groupby('hadm_id').filter(lambda x: len(x) == 9)
len(filtered_df['hadm_id'].unique())

401

In [175]:
filtered_df = test2.groupby('hadm_id').filter(lambda x: len(x) >= 9)

# 각 그룹에서 상위 9개 행만 선택
top_9_df = filtered_df.groupby('hadm_id').head(9)
top_9_df['hadm_id'].nunique()

2116

In [176]:
#test case 6개 만듦듦

test_1, test_2, test_3, test_4, test_5, test_6 = [], [], [], [], [], []

# 각 hadm_id 그룹별로 순차적으로 슬라이싱하여 저장
for _, group in top_9_df.groupby('hadm_id'):
    # 슬라이딩 윈도우 방식으로 1~4, 2~5, ... 6~9번째 데이터 추출
    test_1.append(group.iloc[0:4])  # 1~4번째 데이터
    test_2.append(group.iloc[1:5])  # 2~5번째 데이터
    test_3.append(group.iloc[2:6])  # 3~6번째 데이터
    test_4.append(group.iloc[3:7])  # 4~7번째 데이터
    test_5.append(group.iloc[4:8])  # 5~8번째 데이터
    test_6.append(group.iloc[5:9])  # 6~9번째 데이터

# 각각의 리스트를 데이터프레임으로 변환
test_1 = pd.concat(test_1)
test_2 = pd.concat(test_2)
test_3 = pd.concat(test_3)
test_4 = pd.concat(test_4)
test_5 = pd.concat(test_5)
test_6 = pd.concat(test_6)
test_6

,subject_id,hadm_id,chartdate,SBP_min,SBP_max,SBP_mean,DBP_min,DBP_max,DBP_mean,MAP_min,...,SpO2_max,SpO2_mean,WBC_mean,Hemoglobin_mean,Platelet_mean,Potassium_mean,Chloride_mean,Glucose_mean,Sodium_mean,AKI
18826,14175160,20014600,2163-04-26,133.0,187.0,147.960000,58.0,95.0,67.560000,84.0,...,99.0,95.360000,6.666667,12.60,88.0,3.65,104.0,111.0,148.5,0
18827,14175160,20014600,2163-04-27,137.0,190.0,156.884615,57.0,88.0,70.884615,76.0,...,98.0,95.653846,6.783333,12.60,85.0,3.35,104.5,128.5,153.0,0
18828,14175160,20014600,2163-04-28,119.0,189.0,142.080000,58.0,126.0,76.640000,77.0,...,100.0,96.791667,6.900000,12.60,82.0,3.90,105.0,117.0,150.0,0
18829,14175160,20014600,2163-04-29,113.0,158.0,134.125000,64.0,87.0,75.375000,79.0,...,100.0,97.250000,6.800000,13.50,71.0,3.80,103.5,125.5,149.0,0
42614,19305113,20024364,2129-07-13,119.0,155.0,135.500000,61.0,89.0,70.916667,74.0,...,97.0,93.916667,14.100000,10.30,128.0,4.70,117.0,129.0,152.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6880,11693604,29993948,2123-12-08,106.0,140.0,122.916667,58.0,88.0,74.416667,73.0,...,100.0,96.833333,15.750000,12.25,497.5,3.80,115.5,136.5,148.0,0
32318,17027670,29996046,2170-01-05,95.0,118.0,103.958333,44.0,71.0,56.000000,57.0,...,100.0,99.791667,8.600000,8.20,282.0,4.10,103.0,118.0,135.0,0
32319,17027670,29996046,2170-01-06,93.0,125.0,108.041667,48.0,67.0,56.958333,57.0,...,100.0,99.166667,6.700000,8.30,288.0,4.10,101.0,141.0,134.0,0
32320,17027670,29996046,2170-01-07,93.0,121.0,109.125000,47.0,69.0,58.583333,59.0,...,100.0,99.708333,5.900000,7.90,326.0,5.20,101.0,98.0,134.0,0


In [177]:
test_6['hadm_id'].nunique()

2116

In [178]:
import pandas as pd
import numpy as np

# 슬라이딩 윈도우 생성 함수
def create_sliding_windows_as_df(data, time_steps=2):
    X, y = [], []
    columns = data.drop(columns=['AKI']).columns
    
    for i in range(len(data) - time_steps):
        # X: 현재 타임스텝에 해당하는 윈도우 데이터 (time_steps * 특성 수 크기)
        window_data = data.iloc[i:i + time_steps].drop(columns=['AKI']).values.flatten()
        X.append(window_data)
        
        # y: 현재 윈도우 끝난 후 48시간 이내의 AKI 여부
        y_values = data.iloc[i + time_steps:i + time_steps + 2]['AKI'].values
        y.append(1 if 1 in y_values else 0)
    
    # 컬럼 이름 만들기 (time_steps별로 컬럼에 번호 붙이기)
    flattened_columns = [f"{col}_t{t}" for t in range(1, time_steps + 1) for col in columns]
    
    # X를 데이터프레임으로 변환
    X_df = pd.DataFrame(X, columns=flattened_columns)
    y_df = pd.Series(y, name='AKI')
    
    return X_df, y_df

# 전체 데이터에 대해 슬라이딩 윈도우 적용
X_all, y_all = [], []
for subject_id, group in train2.groupby('hadm_id'):
    group = group.sort_values(by='chartdate')
    X, y = create_sliding_windows_as_df(group)
    if not X.empty:  # 비어 있지 않은 경우만 추가
        X_all.append(X)
        y_all.append(y)

# X_all, y_all을 하나의 데이터프레임으로 연결
X_all = pd.concat(X_all, ignore_index=True)  # shape: (샘플 수, time_steps * 특성 수)
y_all = pd.concat(y_all, ignore_index=True)  # shape: (샘플 수,)

# 최종 데이터프레임 형태 확인
print(X_all.shape)  # (샘플 수, time_steps * 특성 수)
print(y_all.shape)  # (샘플 수,)
# X_all과 y_all을 데이터프레임으로 결합
train2_final = pd.concat([X_all, y_all], axis=1)
train2_final.head()  # 최종 데이터프레임 확인
# (25412, 165)
# (25412,)

# 전체 데이터에 대해 슬라이딩 윈도우 적용
# X_all, y_all = [], []
# for subject_id, group in test2.groupby('hadm_id'):
#     group = group.sort_values(by='chartdate')
#     X, y = create_sliding_windows_as_df(group)
#     if not X.empty:  # 비어 있지 않은 경우만 추가
#         X_all.append(X)
#         y_all.append(y)

# # X_all, y_all을 하나의 데이터프레임으로 연결
# X_all = pd.concat(X_all, ignore_index=True)  # shape: (샘플 수, time_steps * 특성 수)
# y_all = pd.concat(y_all, ignore_index=True)  # shape: (샘플 수,)

# # 최종 데이터프레임 형태 확인
# print(X_all.shape)  # (샘플 수, time_steps * 특성 수)
# print(y_all.shape)  # (샘플 수,)
# # X_all과 y_all을 데이터프레임으로 결합
# test2_final = pd.concat([X_all, y_all], axis=1)
# test2_final.head()  # 최종 데이터프레임 확인
# (25412, 165)
# (25412,)

(83948, 50)
(83948,)


,subject_id_t1,hadm_id_t1,chartdate_t1,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,...,SpO2_max_t2,SpO2_mean_t2,WBC_mean_t2,Hemoglobin_mean_t2,Platelet_mean_t2,Potassium_mean_t2,Chloride_mean_t2,Glucose_mean_t2,Sodium_mean_t2,AKI
0,16230775,20002506,2132-03-19,98.0,114.0,107.055556,53.0,92.0,64.500000,72.0,...,100.0,98.200000,8.0,13.00,174.5,3.95,105.0,96.5,143.333333,0
1,16230775,20002506,2132-03-20,105.0,136.0,118.400000,61.0,80.0,70.200000,74.0,...,100.0,98.750000,9.2,12.75,179.5,3.90,101.0,114.0,140.000000,0
2,16230775,20002506,2132-03-21,110.0,133.0,123.562500,65.0,86.0,73.312500,80.0,...,100.0,98.684211,8.9,12.10,177.0,3.70,95.0,131.0,136.000000,0
3,11725083,20002970,2167-04-27,95.0,170.0,130.863636,53.0,115.0,79.318182,64.0,...,97.0,94.764706,7.3,14.90,303.0,3.50,97.0,164.0,135.000000,0
4,11725083,20002970,2167-04-28,110.0,162.0,133.411765,63.0,105.0,82.294118,81.0,...,96.0,93.857143,15.6,14.10,282.0,3.80,98.0,137.0,134.000000,0


In [179]:
# 빈 리스트로 각 결과 저장
final_dfs = []

# test_1부터 test_6까지 순차적으로 처리
for i, test_df in enumerate([test_1, test_2, test_3, test_4, test_5, test_6], 1):
    X_all, y_all = [], []
    
    # hadm_id별로 슬라이딩 윈도우 생성
    for subject_id, group in test_df.groupby('hadm_id'):
        group = group.sort_values(by='chartdate')  # 날짜 기준으로 정렬
        X, y = create_sliding_windows_as_df(group)  # 슬라이딩 윈도우 생성
        if not X.empty:  # 비어 있지 않은 경우만 추가
            X_all.append(X)
            y_all.append(y)
    
    # X_all, y_all을 하나의 데이터프레임으로 연결
    X_all = pd.concat(X_all, ignore_index=True)  # shape: (샘플 수, time_steps * 특성 수)
    y_all = pd.concat(y_all, ignore_index=True)  # shape: (샘플 수,)
    
    # X_all과 y_all을 데이터프레임으로 결합
    final_df = pd.concat([X_all, y_all], axis=1)
    
    # 각 최종 결과를 리스트에 저장
    final_dfs.append(final_df)
    
    # 최종 데이터프레임 형태 확인
    print(f"test_{i}_final shape: {final_df.shape}")
    
# 결과 데이터프레임 접근 방법: final_dfs[0] -> test_1_final, final_dfs[1] -> test_2_final, ..., final_dfs[5] -> test_6_final

# test_1_final 확인
print(final_dfs[0].head())  # test_1_final


test_1_final shape: (4232, 51)
test_2_final shape: (4232, 51)
test_3_final shape: (4232, 51)
test_4_final shape: (4232, 51)
test_5_final shape: (4232, 51)
test_6_final shape: (4232, 51)
   subject_id_t1  hadm_id_t1 chartdate_t1  SBP_min_t1  SBP_max_t1  \
0       14175160    20014600   2163-04-21       107.0       107.0   
1       14175160    20014600   2163-04-22        92.0       115.0   
2       19305113    20024364   2129-07-07       110.0       163.0   
3       19305113    20024364   2129-07-08        87.0       142.0   
4       19283742    20032690   2149-04-09        65.0       143.0   

   SBP_mean_t1  DBP_min_t1  DBP_max_t1  DBP_mean_t1  MAP_min_t1  ...  \
0   107.000000        77.0        77.0    77.000000        87.0  ...   
1    99.944444        62.0        79.0    70.166667        72.0  ...   
2   139.480000        54.0        94.0    75.440000        79.0  ...   
3   107.884615        55.0        70.0    64.423077        65.0  ...   
4    91.304348        36.0        84.0 

In [180]:
train2_final = train2_final[train2_final.columns[~train2_final.columns.str.contains('subject_id')]]
train2_final = train2_final[train2_final.columns[~train2_final.columns.str.contains('chartdate')]]
train2_final.columns[train2_final.columns.str.contains('hadm_id')][1:]
train2_final.drop(columns=list(train2_final.columns[train2_final.columns.str.contains('hadm_id')][1:]),
                  axis = 1, inplace=True)
train2_final.columns = ['hadm_id', 'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'DBP_min_t1',
       'DBP_max_t1', 'DBP_mean_t1', 'MAP_min_t1', 'MAP_max_t1', 'MAP_mean_t1',
       'HR_min_t1', 'HR_max_t1', 'HR_mean_t1', 'SpO2_min_t1', 'SpO2_max_t1',
       'SpO2_mean_t1', 'WBC_mean_t1', 'Hemoglobin_mean_t1', 'Platelet_mean_t1',
       'Potassium_mean_t1', 'Chloride_mean_t1', 'Glucose_mean_t1',
       'Sodium_mean_t1', 'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2',
       'DBP_min_t2', 'DBP_max_t2', 'DBP_mean_t2', 'MAP_min_t2', 'MAP_max_t2',
       'MAP_mean_t2', 'HR_min_t2', 'HR_max_t2', 'HR_mean_t2', 'SpO2_min_t2',
       'SpO2_max_t2', 'SpO2_mean_t2', 'WBC_mean_t2', 'Hemoglobin_mean_t2',
       'Platelet_mean_t2', 'Potassium_mean_t2', 'Chloride_mean_t2',
       'Glucose_mean_t2', 'Sodium_mean_t2', 'AKI']
train2_final

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,SpO2_max_t2,SpO2_mean_t2,WBC_mean_t2,Hemoglobin_mean_t2,Platelet_mean_t2,Potassium_mean_t2,Chloride_mean_t2,Glucose_mean_t2,Sodium_mean_t2,AKI
0,20002506,98.0,114.0,107.055556,53.0,92.0,64.500000,72.0,99.0,77.055556,...,100.0,98.200000,8.00,13.00,174.5,3.950000,105.000000,96.5,143.333333,0
1,20002506,105.0,136.0,118.400000,61.0,80.0,70.200000,74.0,92.0,83.333333,...,100.0,98.750000,9.20,12.75,179.5,3.900000,101.000000,114.0,140.000000,0
2,20002506,110.0,133.0,123.562500,65.0,86.0,73.312500,80.0,98.0,87.250000,...,100.0,98.684211,8.90,12.10,177.0,3.700000,95.000000,131.0,136.000000,0
3,20002970,95.0,170.0,130.863636,53.0,115.0,79.318182,64.0,128.0,94.772727,...,97.0,94.764706,7.30,14.90,303.0,3.500000,97.000000,164.0,135.000000,0
4,20002970,110.0,162.0,133.411765,63.0,105.0,82.294118,81.0,114.0,95.875000,...,96.0,93.857143,15.60,14.10,282.0,3.800000,98.000000,137.0,134.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83943,29999098,137.0,218.0,178.956522,55.0,94.0,69.260870,82.0,128.0,103.521739,...,100.0,96.448276,9.00,10.60,53.0,3.933333,109.666667,170.0,141.333333,0
83944,29999098,149.0,223.0,190.206897,57.0,91.0,75.827586,84.0,137.0,112.931034,...,98.0,96.448276,10.80,9.50,23.0,3.866667,107.666667,232.0,141.000000,0
83945,29999098,99.0,218.0,158.551724,47.0,92.0,64.793103,66.0,139.0,94.206897,...,100.0,96.875000,13.30,8.10,23.0,3.933333,105.333333,273.5,139.000000,1
83946,29999098,139.0,189.0,164.333333,53.0,81.0,68.666667,71.0,114.0,94.208333,...,100.0,96.666667,15.60,7.10,37.0,3.900000,106.500000,124.5,139.500000,1


In [181]:
train2_final = pd.merge(train2_final,d_onebyone_table_train,how='left',on='hadm_id')
# train2_final.drop(columns=['hadm_id'],axis = 1, inplace= True)
train2_final

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,Glucose_mean_t2,Sodium_mean_t2,AKI,medicine,weight,diagnose,height,gender,anchor_age,race
0,20002506,98.0,114.0,107.055556,53.0,92.0,64.500000,72.0,99.0,77.055556,...,96.5,143.333333,0,1,60.0,0,174.298920,0,24,UNKNOWN
1,20002506,105.0,136.0,118.400000,61.0,80.0,70.200000,74.0,92.0,83.333333,...,114.0,140.000000,0,1,60.0,0,174.298920,0,24,UNKNOWN
2,20002506,110.0,133.0,123.562500,65.0,86.0,73.312500,80.0,98.0,87.250000,...,131.0,136.000000,0,1,60.0,0,174.298920,0,24,UNKNOWN
3,20002970,95.0,170.0,130.863636,53.0,115.0,79.318182,64.0,128.0,94.772727,...,164.0,135.000000,0,1,57.9,0,158.306376,1,75,WHITE
4,20002970,110.0,162.0,133.411765,63.0,105.0,82.294118,81.0,114.0,95.875000,...,137.0,134.000000,0,1,57.9,0,158.306376,1,75,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83943,29999098,137.0,218.0,178.956522,55.0,94.0,69.260870,82.0,128.0,103.521739,...,170.0,141.333333,0,1,91.6,0,167.640000,0,76,WHITE
83944,29999098,149.0,223.0,190.206897,57.0,91.0,75.827586,84.0,137.0,112.931034,...,232.0,141.000000,0,1,91.6,0,167.640000,0,76,WHITE
83945,29999098,99.0,218.0,158.551724,47.0,92.0,64.793103,66.0,139.0,94.206897,...,273.5,139.000000,1,1,91.6,0,167.640000,0,76,WHITE
83946,29999098,139.0,189.0,164.333333,53.0,81.0,68.666667,71.0,114.0,94.208333,...,124.5,139.500000,1,1,91.6,0,167.640000,0,76,WHITE


In [182]:
import pandas as pd

# 데이터프레임을 처리하는 함수 정의
def process_final_dataframe(df):
    # subject_id와 chartdate가 포함된 열 제거
    df = df[df.columns[~df.columns.str.contains('subject_id')]]
    df = df[df.columns[~df.columns.str.contains('chartdate')]]
    
    # 중복된 hadm_id 열 제거 (첫 번째만 남기고 제거)
    df.drop(columns=list(df.columns[df.columns.str.contains('hadm_id')][1:]), axis=1, inplace=True)
    
    # 열 이름 재설정
    df.columns = ['hadm_id', 'SBP_min_t1', 'SBP_max_t1', 'SBP_mean_t1', 'DBP_min_t1',
                  'DBP_max_t1', 'DBP_mean_t1', 'MAP_min_t1', 'MAP_max_t1', 'MAP_mean_t1',
                  'HR_min_t1', 'HR_max_t1', 'HR_mean_t1', 'SpO2_min_t1', 'SpO2_max_t1',
                  'SpO2_mean_t1', 'WBC_mean_t1', 'Hemoglobin_mean_t1', 'Platelet_mean_t1',
                  'Potassium_mean_t1', 'Chloride_mean_t1', 'Glucose_mean_t1',
                  'Sodium_mean_t1', 'SBP_min_t2', 'SBP_max_t2', 'SBP_mean_t2',
                  'DBP_min_t2', 'DBP_max_t2', 'DBP_mean_t2', 'MAP_min_t2', 'MAP_max_t2',
                  'MAP_mean_t2', 'HR_min_t2', 'HR_max_t2', 'HR_mean_t2', 'SpO2_min_t2',
                  'SpO2_max_t2', 'SpO2_mean_t2', 'WBC_mean_t2', 'Hemoglobin_mean_t2',
                  'Platelet_mean_t2', 'Potassium_mean_t2', 'Chloride_mean_t2',
                  'Glucose_mean_t2', 'Sodium_mean_t2', 'AKI']
    return df

# 각 test_final 데이터프레임에 함수 적용
final_dfs_processed = [process_final_dataframe(df) for df in final_dfs]

final_dfs_processed[0]

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,SpO2_max_t2,SpO2_mean_t2,WBC_mean_t2,Hemoglobin_mean_t2,Platelet_mean_t2,Potassium_mean_t2,Chloride_mean_t2,Glucose_mean_t2,Sodium_mean_t2,AKI
0,20014600,107.0,107.0,107.000000,77.0,77.0,77.000000,87.0,87.0,87.000000,...,100.0,97.541667,6.200000,12.600000,100.000000,3.966667,103.333333,117.0,140.333333,0
1,20014600,92.0,115.0,99.944444,62.0,79.0,70.166667,72.0,91.0,80.388889,...,99.0,96.000000,6.316667,12.600000,97.000000,4.050000,106.000000,118.5,145.000000,0
2,20024364,110.0,163.0,139.480000,54.0,94.0,75.440000,79.0,106.0,93.240000,...,98.0,93.846154,11.000000,12.400000,220.000000,3.800000,108.500000,96.0,141.000000,1
3,20024364,87.0,142.0,107.884615,55.0,70.0,64.423077,65.0,90.0,76.961538,...,99.0,89.609756,10.250000,11.050000,140.500000,3.900000,111.000000,103.5,142.000000,1
4,20032690,65.0,143.0,91.304348,36.0,84.0,60.285714,37.0,89.0,64.739130,...,99.0,53.250000,28.250000,9.200000,193.000000,3.900000,88.600000,302.8,133.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,29989470,100.0,160.0,124.880000,51.0,73.0,63.280000,67.0,100.0,83.200000,...,100.0,98.846154,7.733333,6.966667,194.666667,4.350000,118.500000,141.5,145.500000,0
4228,29993948,84.0,172.0,126.411765,41.0,97.0,69.058824,45.0,123.0,87.960784,...,100.0,98.708333,16.500000,12.300000,285.500000,3.100000,117.500000,134.0,146.400000,0
4229,29993948,98.0,152.0,121.689655,60.0,105.0,74.310345,69.0,109.0,86.758621,...,99.0,97.833333,16.650000,11.250000,265.500000,3.400000,116.000000,113.0,147.750000,0
4230,29996046,89.0,138.0,116.933333,46.0,72.0,57.800000,55.0,85.0,71.400000,...,100.0,98.916667,18.900000,11.000000,293.000000,3.500000,105.000000,112.0,136.000000,0


In [183]:
# 병합 후 hadm_id 열을 제거하는 함수 정의
def merge_and_process(df, merge_df):
    # df와 merge_df를 hadm_id 기준으로 left join
    merged_df = pd.merge(df, merge_df, how='left', on='hadm_id')
    
    # 병합 후 hadm_id 열을 제거할지 선택 (주석 해제 시 제거)
    # merged_df.drop(columns=['hadm_id'], axis=1, inplace=True)
    
    return merged_df

# final_dfs_processed 리스트의 각 데이터프레임에 대해 병합 처리
final_dfs_merged = [merge_and_process(df, d_onebyone_table_test) for df in final_dfs_processed]

# 결과 확인 (예: 첫 번째 데이터프레임 확인)
print(final_dfs_merged[0].head())


    hadm_id  SBP_min_t1  SBP_max_t1  SBP_mean_t1  DBP_min_t1  DBP_max_t1  \
0  20014600       107.0       107.0   107.000000        77.0        77.0   
1  20014600        92.0       115.0    99.944444        62.0        79.0   
2  20024364       110.0       163.0   139.480000        54.0        94.0   
3  20024364        87.0       142.0   107.884615        55.0        70.0   
4  20032690        65.0       143.0    91.304348        36.0        84.0   

   DBP_mean_t1  MAP_min_t1  MAP_max_t1  MAP_mean_t1  ...  Glucose_mean_t2  \
0    77.000000        87.0        87.0    87.000000  ...            117.0   
1    70.166667        72.0        91.0    80.388889  ...            118.5   
2    75.440000        79.0       106.0    93.240000  ...             96.0   
3    64.423077        65.0        90.0    76.961538  ...            103.5   
4    60.285714        37.0        89.0    64.739130  ...            302.8   

   Sodium_mean_t2  AKI  medicine  weight  diagnose  height  gender  \
0      140

In [184]:
num_col_to_scale = ['weight', 'height'] + [col for col in train2_final.columns if 't1' in col or 't2' in col]
cat_col = ['medicine','diagnose']
target_col = ['AKI']

from sklearn.preprocessing import StandardScaler

# 스케일러 인스턴스 생성
scaler = StandardScaler()
train2_final[num_col_to_scale] = scaler.fit_transform(train2_final[num_col_to_scale])

# 테스트 데이터 스케일링
# 주의: 테스트 데이터는 'fit'하지 않고 'transform'만 실행
for i in range(len(final_dfs_merged)):
    final_dfs_merged[i][num_col_to_scale] = scaler.transform(final_dfs_merged[i][num_col_to_scale])

# 결과 확인 (예: 첫 번째 데이터프레임 확인)
final_dfs_merged[0]

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,Glucose_mean_t2,Sodium_mean_t2,AKI,medicine,weight,diagnose,height,gender,anchor_age,race
0,20014600,0.653512,-1.708390,-0.754596,2.632294,-0.400719,1.202511,1.955551,-0.772233,0.627867,...,-0.465797,0.150455,0,1,0.756830,1,0.596174,0,55,WHITE
1,20014600,-0.211363,-1.372416,-1.151531,1.239548,-0.300974,0.611532,0.747780,-0.568455,0.067878,...,-0.433869,1.026151,0,1,0.756830,1,0.596174,0,55,WHITE
2,20024364,0.826487,0.643430,1.072677,0.496751,0.447109,1.067595,1.311407,0.195713,1.156421,...,-0.912790,0.275554,1,1,0.737192,1,1.329989,0,51,WHITE
3,20024364,-0.499654,-0.238502,-0.704829,0.589600,-0.749824,0.114800,0.184153,-0.619400,-0.222432,...,-0.753150,0.463204,1,1,0.737192,1,1.329989,0,51,WHITE
4,20032690,-1.768137,-0.196506,-1.637609,-1.174544,-0.051613,-0.243019,-2.070353,-0.670344,-1.257721,...,3.489030,-1.131813,0,1,-0.912437,1,-0.626852,0,61,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,29989470,0.249904,0.517440,0.251304,0.218201,-0.600207,0.015941,0.345190,-0.109954,0.305991,...,0.055695,1.119975,0,1,-0.413621,3,-1.177213,1,64,BLACK/AFRICAN AMERICAN
4228,29993948,-0.672629,1.021401,0.337478,-0.710296,0.596725,0.515721,-1.426208,1.061771,0.709249,...,-0.103945,1.288859,0,1,-0.888871,0,-1.360667,1,60,WHITE
4229,29993948,0.134587,0.181466,0.071820,1.053849,0.995703,0.969897,0.506226,0.348547,0.607421,...,-0.550939,1.542186,0,1,-0.888871,0,-1.360667,1,60,WHITE
4230,29996046,-0.384338,-0.406490,-0.195763,-0.246047,-0.650080,-0.457996,-0.621027,-0.874122,-0.693518,...,-0.572224,-0.662691,0,1,-0.378272,0,-0.382247,1,70,BLACK/CAPE VERDEAN


In [185]:
set(train2_final[cat_col[0]]) # {0, 1}
set(train2_final[cat_col[1]]) # {0, 1, 2, 3}
print(train2_final.shape)
train2_final.head()

(83948, 53)


,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,Glucose_mean_t2,Sodium_mean_t2,AKI,medicine,weight,diagnose,height,gender,anchor_age,race
0,20002506,0.134587,-1.414413,-0.751471,0.403901,0.347364,0.121452,0.747780,-0.160899,-0.214469,...,-0.902148,0.713402,0,1,-0.971353,0,0.503621,0,24,UNKNOWN
1,20002506,0.538195,-0.490483,-0.113251,1.146699,-0.251102,0.614415,0.908816,-0.517510,0.317285,...,-0.529653,0.087905,0,1,-0.971353,0,0.503621,0,24,UNKNOWN
2,20002506,0.826487,-0.616474,0.177183,1.518098,0.048131,0.883599,1.391925,-0.211843,0.649043,...,-0.167801,-0.662691,0,1,-0.971353,0,0.503621,0,24,UNKNOWN
3,20002970,-0.038388,0.937408,0.587934,0.403901,1.494425,1.402999,0.103635,1.316494,1.286249,...,0.534617,-0.850340,0,1,-1.053834,0,-1.036481,1,75,WHITE
4,20002970,0.826487,0.601433,0.731287,1.332398,0.995703,1.660372,1.472443,0.603270,1.379616,...,-0.040089,-1.037989,0,1,-1.053834,0,-1.036481,1,75,WHITE


In [186]:
train_encoded = pd.get_dummies(train2_final, columns=['diagnose'], prefix='diag')
#final_train_encoded, final_test_encoded = train_encoded.align(final_dfs_merged[0], join='outer', axis=1, fill_value=0)
train_encoded[['diag_0',	'diag_1',	'diag_2',	'diag_3']] = train_encoded[['diag_0',	'diag_1',	'diag_2',	'diag_3']].astype(int)
# 각 test 데이터프레임에 대해 반복
encoded_dfs = []  # 인코딩된 데이터프레임을 저장할 리스트

for i in range(6):  # 0부터 5까지 반복 (test1부터 test6까지)
    test_encoded = pd.get_dummies(final_dfs_merged[i], columns=['diagnose'], prefix='diag')
    test_encoded[['diag_0', 'diag_1', 'diag_2', 'diag_3']] = test_encoded[['diag_0', 'diag_1', 'diag_2', 'diag_3']].astype(int)
    encoded_dfs.append(test_encoded)  # 인코딩된 데이터프레임을 리스트에 추가

# 이제 encoded_dfs[0]은 test1_encoded, encoded_dfs[1]은 test2_encoded 등입니다.
encoded_dfs[0]

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,medicine,weight,height,gender,anchor_age,race,diag_0,diag_1,diag_2,diag_3
0,20014600,0.653512,-1.708390,-0.754596,2.632294,-0.400719,1.202511,1.955551,-0.772233,0.627867,...,1,0.756830,0.596174,0,55,WHITE,0,1,0,0
1,20014600,-0.211363,-1.372416,-1.151531,1.239548,-0.300974,0.611532,0.747780,-0.568455,0.067878,...,1,0.756830,0.596174,0,55,WHITE,0,1,0,0
2,20024364,0.826487,0.643430,1.072677,0.496751,0.447109,1.067595,1.311407,0.195713,1.156421,...,1,0.737192,1.329989,0,51,WHITE,0,1,0,0
3,20024364,-0.499654,-0.238502,-0.704829,0.589600,-0.749824,0.114800,0.184153,-0.619400,-0.222432,...,1,0.737192,1.329989,0,51,WHITE,0,1,0,0
4,20032690,-1.768137,-0.196506,-1.637609,-1.174544,-0.051613,-0.243019,-2.070353,-0.670344,-1.257721,...,1,-0.912437,-0.626852,0,61,WHITE,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,29989470,0.249904,0.517440,0.251304,0.218201,-0.600207,0.015941,0.345190,-0.109954,0.305991,...,1,-0.413621,-1.177213,1,64,BLACK/AFRICAN AMERICAN,0,0,0,1
4228,29993948,-0.672629,1.021401,0.337478,-0.710296,0.596725,0.515721,-1.426208,1.061771,0.709249,...,1,-0.888871,-1.360667,1,60,WHITE,1,0,0,0
4229,29993948,0.134587,0.181466,0.071820,1.053849,0.995703,0.969897,0.506226,0.348547,0.607421,...,1,-0.888871,-1.360667,1,60,WHITE,1,0,0,0
4230,29996046,-0.384338,-0.406490,-0.195763,-0.246047,-0.650080,-0.457996,-0.621027,-0.874122,-0.693518,...,1,-0.378272,-0.382247,1,70,BLACK/CAPE VERDEAN,1,0,0,0


In [187]:
X_tests = []  # X_test를 저장할 리스트
y_tests = []  # y_test를 저장할 리스트

for i in range(6):  # 0부터 5까지 반복 (test1부터 test6까지)
    encoded_df = encoded_dfs[i]  # 인코딩된 데이터프레임 선택
    X_test = encoded_df.drop(columns=['AKI'])  # AKI 열을 제외한 피처
    y_test = encoded_df['AKI']  # AKI 열
    
    X_tests.append(X_test)  # X_test를 리스트에 추가
    y_tests.append(y_test)  # y_test를 리스트에 추가

In [188]:
X_tests[0]

,hadm_id,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,...,medicine,weight,height,gender,anchor_age,race,diag_0,diag_1,diag_2,diag_3
0,20014600,0.653512,-1.708390,-0.754596,2.632294,-0.400719,1.202511,1.955551,-0.772233,0.627867,...,1,0.756830,0.596174,0,55,WHITE,0,1,0,0
1,20014600,-0.211363,-1.372416,-1.151531,1.239548,-0.300974,0.611532,0.747780,-0.568455,0.067878,...,1,0.756830,0.596174,0,55,WHITE,0,1,0,0
2,20024364,0.826487,0.643430,1.072677,0.496751,0.447109,1.067595,1.311407,0.195713,1.156421,...,1,0.737192,1.329989,0,51,WHITE,0,1,0,0
3,20024364,-0.499654,-0.238502,-0.704829,0.589600,-0.749824,0.114800,0.184153,-0.619400,-0.222432,...,1,0.737192,1.329989,0,51,WHITE,0,1,0,0
4,20032690,-1.768137,-0.196506,-1.637609,-1.174544,-0.051613,-0.243019,-2.070353,-0.670344,-1.257721,...,1,-0.912437,-0.626852,0,61,WHITE,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,29989470,0.249904,0.517440,0.251304,0.218201,-0.600207,0.015941,0.345190,-0.109954,0.305991,...,1,-0.413621,-1.177213,1,64,BLACK/AFRICAN AMERICAN,0,0,0,1
4228,29993948,-0.672629,1.021401,0.337478,-0.710296,0.596725,0.515721,-1.426208,1.061771,0.709249,...,1,-0.888871,-1.360667,1,60,WHITE,1,0,0,0
4229,29993948,0.134587,0.181466,0.071820,1.053849,0.995703,0.969897,0.506226,0.348547,0.607421,...,1,-0.888871,-1.360667,1,60,WHITE,1,0,0,0
4230,29996046,-0.384338,-0.406490,-0.195763,-0.246047,-0.650080,-0.457996,-0.621027,-0.874122,-0.693518,...,1,-0.378272,-0.382247,1,70,BLACK/CAPE VERDEAN,1,0,0,0


In [189]:
# 양성 클래스와 음성 클래스의 비율 계산
ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)
ratio

3.86824402690791

In [190]:
ratio2 = len(y_train) / sum(y_train) 
ratio2

4.86824402690791

In [192]:
model_xgb_pre = xgb.XGBClassifier(random_state=42,
                              # scale_pos_weight=ratio,
                              scale_pos_weight=recall_85['scale_pos_weight'],
                              max_depth = recall_85['max_depth'],
                              n_estimators= recall_85['n_estimators'],
                              alpha = recall_85['alpha'],
                              gamma = recall_85['gamma'],
                              learning_rate = recall_85['learning_rate'],
                             ) 

model_xgb_pre.fit(X_train, y_train)


XGBClassifier(alpha=1.5047452166545056e-05, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1.5482527880396597e-06, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01276320292990235, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=476, n_jobs=None,
              num_parallel_tree=None, ...)

In [193]:
# X_train의 칼럼 목록 가져오기
train_columns = set(X_train.columns)

# 각 테스트 데이터프레임에서 X_train에 없는 칼럼 제거
for i in range(6):  # 0부터 5까지 반복 (test1부터 test6까지)
    X_tests[i] = X_tests[i][X_tests[i].columns.intersection(train_columns)]  # X_train에 있는 칼럼만 선택
X_tests[0]

,SBP_min_t1,SBP_max_t1,SBP_mean_t1,DBP_min_t1,DBP_max_t1,DBP_mean_t1,MAP_min_t1,MAP_max_t1,MAP_mean_t1,HR_min_t1,...,Chloride_mean_t2,Glucose_mean_t2,Sodium_mean_t2,medicine,weight,height,diag_0,diag_1,diag_2,diag_3
0,0.653512,-1.708390,-0.754596,2.632294,-0.400719,1.202511,1.955551,-0.772233,0.627867,0.190476,...,-0.001674,-0.465797,0.150455,1,0.756830,0.596174,0,1,0,0
1,-0.211363,-1.372416,-1.151531,1.239548,-0.300974,0.611532,0.747780,-0.568455,0.067878,-0.395281,...,0.403878,-0.433869,1.026151,1,0.756830,0.596174,0,1,0,0
2,0.826487,0.643430,1.072677,0.496751,0.447109,1.067595,1.311407,0.195713,1.156421,1.882663,...,0.784083,-0.912790,0.275554,1,0.737192,1.329989,0,1,0,0
3,-0.499654,-0.238502,-0.704829,0.589600,-0.749824,0.114800,0.184153,-0.619400,-0.222432,-0.069860,...,1.164288,-0.753150,0.463204,1,0.737192,1.329989,0,1,0,0
4,-1.768137,-0.196506,-1.637609,-1.174544,-0.051613,-0.243019,-2.070353,-0.670344,-1.257721,-1.306458,...,-2.242350,3.489030,-1.131813,1,-0.912437,-0.626852,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,0.249904,0.517440,0.251304,0.218201,-0.600207,0.015941,0.345190,-0.109954,0.305991,-1.436626,...,2.304903,0.055695,1.119975,1,-0.413621,-1.177213,0,0,0,1
4228,-0.672629,1.021401,0.337478,-0.710296,0.596725,0.515721,-1.426208,1.061771,0.709249,-0.850869,...,2.152821,-0.103945,1.288859,1,-0.888871,-1.360667,1,0,0,0
4229,0.134587,0.181466,0.071820,1.053849,0.995703,0.969897,0.506226,0.348547,0.607421,-0.330196,...,1.924698,-0.550939,1.542186,1,-0.888871,-1.360667,1,0,0,0
4230,-0.384338,-0.406490,-0.195763,-0.246047,-0.650080,-0.457996,-0.621027,-0.874122,-0.693518,0.646065,...,0.251796,-0.572224,-0.662691,1,-0.378272,-0.382247,1,0,0,0


In [194]:
# 각 모델에 대한 예측 및 평가 수행
for i in range(6):  # 0부터 5까지 반복 (test1부터 test6까지)
    X_test = X_tests[i]  # 각 테스트 데이터
    y_test = y_tests[i]  # 각 테스트 레이블
    
    pred_xgb = model_xgb_pre.predict(X_test)  # 예측 수행
    print(f'Case {i + 1} Results:')
    get_clf_eval(y_test, pred_xgb)  # 평가 함수 호출
    print('\n')


Case 1 Results:
오차행렬:
 [[1588 1807]
 [ 104  733]]

정확도: 0.5484
정밀도(precision 0.2 목표): 0.2886
재현율(recall 0.75 목표): 0.8757
F1: 0.4341


Case 2 Results:
오차행렬:
 [[1621 1877]
 [ 100  634]]

정확도: 0.5328
정밀도(precision 0.2 목표): 0.2525
재현율(recall 0.75 목표): 0.8638
F1: 0.3908


Case 3 Results:
오차행렬:
 [[1674 1911]
 [  82  565]]

정확도: 0.5291
정밀도(precision 0.2 목표): 0.2282
재현율(recall 0.75 목표): 0.8733
F1: 0.3618


Case 4 Results:
오차행렬:
 [[1699 1951]
 [  75  507]]

정확도: 0.5213
정밀도(precision 0.2 목표): 0.2063
재현율(recall 0.75 목표): 0.8711
F1: 0.3336


Case 5 Results:
오차행렬:
 [[1647 1725]
 [ 118  742]]

정확도: 0.5645
정밀도(precision 0.2 목표): 0.3008
재현율(recall 0.75 목표): 0.8628
F1: 0.4460


Case 6 Results:
오차행렬:
 [[1647 1722]
 [ 109  754]]

정확도: 0.5673
정밀도(precision 0.2 목표): 0.3045
재현율(recall 0.75 목표): 0.8737
F1: 0.4516


